<a href="https://colab.research.google.com/github/AndikaPutra509/Prediksi-Saham/blob/main/Model_Porto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import copy
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import hashlib
from scipy import stats
from scipy.signal import find_peaks
import pickle
import os
import time
import json
from collections import deque

# ===================== BOOTSTRAP DEPENDENCIES =====================
def ensure_package(pkg: str):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

required_packages = [
    "ta", "yfinance", "tqdm", "tabulate", "scipy",
    "xgboost", "statsmodels", "arch"
]

for pkg in required_packages:
    ensure_package(pkg)

import yfinance as yf
import ta
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.exceptions import ConvergenceWarning
from tabulate import tabulate
import xgboost as xgb
import statsmodels.api as sm
from arch import arch_model

# Ignore warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

SEED = 42
np.random.seed(SEED)

# ===================== THREAD SAFETY LOCKS =====================
global_cache_lock = threading.RLock()
results_lock = threading.Lock()
debug_lock = threading.Lock()
trade_history_lock = threading.Lock()

# ===================== SUPPRESS YFINANCE ERRORS =====================
import logging
logging.getLogger("yfinance").setLevel(logging.CRITICAL)

# ===================== KONFIGURASI =====================
MODAL = float(input("Masukkan modal Anda (Rp): "))
INTERVAL = "1h"
JKT_TZ = "Asia/Jakarta"
SESSION1_START = "09:00:00"
SESSION1_END = "12:00:00"
SESSION2_START = "13:30:00"
SESSION2_END = "16:00:00"

# Filter dasar
MIN_HISTORY_BARS = 50

# ===================== DEBUG MODE =====================
DEBUG_MODE = False
DEBUG_SAMPLE = 50

# ===================== FILTER HARGA ADAPTIF =====================
def get_price_range_based_on_modal(modal):
    """
    Menentukan range harga saham yang sesuai berdasarkan modal.
    """
    if modal < 100000:  # Modal < 100rb
        min_price = 10
        max_price = 500
        price_category = "RENDAH (Rp 10-500)"
        base_min_volume = 50000
        base_max_spread = 10.0
    elif modal < 500000:  # Modal 100rb - 500rb
        min_price = 10
        max_price = 1000
        price_category = "RENDAH-MENENGAH (Rp 10-1000)"
        base_min_volume = 100000
        base_max_spread = 9.0
    elif modal < 2000000:  # Modal 500rb - 2jt
        min_price = 10
        max_price = 5000
        price_category = "MENENGAH (Rp 10-5000)"
        base_min_volume = 200000
        base_max_spread = 8.0
    elif modal < 10000000:  # Modal 2jt - 10jt
        min_price = 500
        max_price = 10000
        price_category = "MENENGAH-TINGGI (Rp 500-10000)"
        base_min_volume = 500000
        base_max_spread = 6.0
    else:  # Modal > 10jt
        min_price = 1000
        max_price = 20000
        price_category = "TINGGI (Rp 1000-20000)"
        base_min_volume = 1000000
        base_max_spread = 5.0

    return min_price, max_price, price_category, base_min_volume, base_max_spread

# Set filter berdasarkan modal
MIN_PRICE, MAX_PRICE, PRICE_CATEGORY, BASE_MIN_VOLUME, BASE_MAX_SPREAD = get_price_range_based_on_modal(MODAL)

# FEE TRANSAKSI
FEE_BUY = 0.0015  # 0.15% beli
FEE_SELL = 0.0015  # 0.15% jual
TOTAL_FEE = FEE_BUY + FEE_SELL  # 0.3%

# Threshold
TARGET_HORIZON_BARS = 8
RETURN_THRESHOLD = 0.01

# Risk Parameters
MAX_POSITION_SIZE = 0.50 if MODAL < 100000 else 0.30 if MODAL < 500000 else 0.20
MIN_POSITION_SIZE = 0.05
RISK_PER_TRADE = 0.02

# Support/Resistance Parameters
SR_LOOKBACK = 100
SR_PROMINENCE = 0.02

MIN_RR_RATIO = 1.5  # Setelah fee
USE_PROB_CALIBRATION = True
N_SPLITS = 5

# Counter untuk debug
debug_counter = 0

# Cache untuk feature engineering
_feature_cache = {}

# ===================== KONFIGURASI INDEKS GLOBAL =====================
GLOBAL_INDICES = {
    "IHSG": "^JKSE",
    "SHANGHAI": "000001.SS",
    "DOWJONES": "^DJI",
    "S&P500": "^GSPC",
    "NASDAQ": "^IXIC",
    "NIKKEI": "^N225",
    "OIL": "CL=F",
    "GOLD": "GC=F",
    "USDIDR": "IDR=X",
    "HANGSENG": "^HSI",
    "CAC40": "^FCHI",
    "DAX": "^GDAXI",
    "BRENT": "BZ=F",
    "COAL": "MTF=F",
    "CPO": "FCPOc1",
    "SILVER": "SI=F",
    "NICKEL": "NI=F",
}

# ===================== GLOBAL INDICES CACHE =====================
_global_indices_cache = None
_global_indices_series_cache = {}

def get_global_indices_data(force_refresh=False):
    """Mendapatkan data indeks global dengan cache permanent."""
    global _global_indices_cache

    if _global_indices_cache is not None and not force_refresh:
        return _global_indices_cache

    print("📡 Mengunduh data indeks global (sekali saja)...")
    indices_data = {}

    for name, ticker in GLOBAL_INDICES.items():
        try:
            series = download_index_data(ticker)
            if series is not None and len(series) > 0:
                current = float(series.iloc[-1])
                prev = float(series.iloc[-2]) if len(series) > 1 else current
                change = ((current - prev) / prev) * 100

                # Momentum (5 periode)
                if len(series) > 5:
                    momentum = (series.iloc[-1] / series.iloc[-6] - 1) * 100
                else:
                    momentum = 0

                # Trend
                if len(series) > 5:
                    ma5 = series.tail(5).mean()
                    trend = "🟢 BULLISH" if current > ma5 else "🔴 BEARISH" if current < ma5 else "🟡 NETRAL"
                else:
                    trend = "🟡 NETRAL"

                # Simpan series untuk korelasi
                returns = series.pct_change().dropna()

                indices_data[name] = {
                    'value': current,
                    'change': change,
                    'momentum': momentum,
                    'trend': trend,
                    'returns': returns.tail(100) if len(returns) > 100 else returns,
                }

                with global_cache_lock:
                    _global_indices_series_cache[name] = series
            else:
                indices_data[name] = {
                    'value': 0, 'change': 0, 'momentum': 0,
                    'trend': "⚪ TIDAK ADA DATA",
                    'returns': pd.Series()
                }
        except Exception as e:
            indices_data[name] = {
                'value': 0, 'change': 0, 'momentum': 0,
                'trend': "⚪ TIDAK ADA DATA",
                'returns': pd.Series()
            }

    _global_indices_cache = indices_data
    return indices_data

def download_index_data(ticker: str) -> pd.Series | None:
    try:
        raw = yf.download(ticker, period="6mo", interval=INTERVAL, auto_adjust=True, progress=False, timeout=5)
        if raw.empty:
            return None
        raw = normalize_columns(raw)
        return raw["Close"]
    except Exception:
        return None

def add_global_indices_features_optimized(df: pd.DataFrame) -> pd.DataFrame:
    """
    Menambahkan fitur global ke dataframe.
    """
    out = df.copy()

    with global_cache_lock:
        cache_items = list(_global_indices_series_cache.items())

    for name, series in cache_items:
        try:
            if series is None or len(series) == 0:
                continue

            series_aligned = series.reindex(out.index, method='ffill')

            ret_1 = series_aligned.pct_change().fillna(0)
            ret_5 = series_aligned.pct_change(5).fillna(0)
            ret_20 = series_aligned.pct_change(20).fillna(0)

            out[f"{name}_Return_1"] = ret_1
            out[f"{name}_Return_5"] = ret_5
            out[f"{name}_Return_20"] = ret_20
            out[f"{name}_Momentum"] = (series_aligned / series_aligned.rolling(20).mean() - 1).fillna(0)
            out[f"{name}_Volatility"] = ret_1.rolling(10).std().fillna(0)
        except Exception as e:
            continue

    return out

# ===================== FUNGSI UTILITAS =====================
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df

def replace_inf_with_nan(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    num_cols = out.select_dtypes(include=[np.number]).columns
    out.loc[:, num_cols] = out.loc[:, num_cols].replace([np.inf, -np.inf], np.nan)
    return out

def apply_idx_to_jakarta(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    if not isinstance(out.index, pd.DatetimeIndex):
        return out
    if out.index.tz is None:
        out.index = out.index.tz_localize(JKT_TZ)
    else:
        out.index = out.index.tz_convert(JKT_TZ)
    return out

def filter_jakarta_sessions(df: pd.DataFrame) -> pd.DataFrame:
    out = apply_idx_to_jakarta(df)
    sess1 = out.between_time(SESSION1_START, SESSION1_END, inclusive="both")
    sess2 = out.between_time(SESSION2_START, SESSION2_END, inclusive="both")
    out = pd.concat([sess1, sess2]).sort_index()
    if isinstance(out.index, pd.DatetimeIndex):
        out = out[out.index.dayofweek < 5]
    return out

def download_stock_data(symbol: str) -> pd.DataFrame | None:
    try:
        raw = yf.download(symbol, period="6mo", interval=INTERVAL, auto_adjust=True, progress=False, timeout=5)
        if raw.empty:
            return None
        raw = normalize_columns(raw)
        raw = filter_jakarta_sessions(raw)
        if len(raw) < MIN_HISTORY_BARS:
            return None
        return raw
    except Exception:
        return None

def calculate_spread_pct(df: pd.DataFrame) -> float:
    try:
        avg_spread = ((df["High"] - df["Low"]) / df["Close"]).tail(20).mean() * 100
        return round(avg_spread, 2)
    except:
        return 999.0

def get_adaptive_min_volume(price: float) -> float:
    if price < 100:
        return max(BASE_MIN_VOLUME, 50000)
    elif price < 500:
        return max(BASE_MIN_VOLUME, 100000)
    elif price < 1000:
        return max(BASE_MIN_VOLUME, 200000)
    elif price < 5000:
        return max(BASE_MIN_VOLUME, 500000)
    else:
        return max(BASE_MIN_VOLUME, 1000000)

def get_adaptive_max_spread(price: float) -> float:
    if price < 100:
        return max(BASE_MAX_SPREAD, 12.0)
    elif price < 500:
        return max(BASE_MAX_SPREAD, 10.0)
    elif price < 1000:
        return max(BASE_MAX_SPREAD, 8.0)
    else:
        return BASE_MAX_SPREAD

def get_next_bej_datetimes_2days(start_date: datetime) -> list:
    results = []
    current = start_date

    if current.tzinfo is None:
        current = pd.Timestamp(current).tz_localize(JKT_TZ)
    else:
        current = pd.Timestamp(current).tz_convert(JKT_TZ)

    session_hours = [9, 10, 11, 12, 14, 15, 16]

    for hour in range(1, 15):
        target_hours = hour
        temp_date = current

        hours_added = 0
        while hours_added < target_hours:
            temp_date = temp_date + timedelta(hours=1)

            while temp_date.dayofweek >= 5:
                temp_date = temp_date + timedelta(days=1)
                temp_date = temp_date.replace(hour=9, minute=0, second=0)

            if temp_date.hour in session_hours:
                hours_added += 1

        day_indonesia = {
            'Monday': 'Senin', 'Tuesday': 'Selasa', 'Wednesday': 'Rabu',
            'Thursday': 'Kamis', 'Friday': 'Jumat', 'Saturday': 'Sabtu',
            'Sunday': 'Minggu'
        }
        day_en = temp_date.strftime("%A")
        day_id = day_indonesia.get(day_en, day_en)

        if temp_date.hour in [9, 10, 11, 12]:
            sesi = "Sesi 1"
        else:
            sesi = "Sesi 2"

        if hour <= 7:
            day_num = 1
        else:
            day_num = 2

        results.append({
            "jam_ke": hour,
            "hari_ke": day_num,
            "datetime": temp_date.strftime("%Y-%m-%d %H:%M"),
            "tanggal": temp_date.strftime("%Y-%m-%d"),
            "jam": temp_date.strftime("%H:%M"),
            "hari": day_id,
            "sesi": sesi,
            "date_obj": temp_date
        })

    return results

# ===================== FUNGSI SEKTOR =====================
def get_sector_from_symbol(symbol: str) -> str:
    symbol_clean = symbol.replace(".JK", "")

    if symbol_clean in ["ADRO", "PTBA", "ITMG", "MEDC", "PGAS", "ENRG", "BUMI", "DOID"]:
        return "Energy"
    if symbol_clean in ["ANTM", "INCO", "TINS", "HRUM", "MDKA", "BRMS", "NIKL", "CITA"]:
        return "Mining"
    if symbol_clean in ["AALI", "LSIP", "SGRO", "BWPT", "GZCO", "DSNG", "JAWA", "SIMP", "GOOD"]:
        return "Agriculture"
    if symbol_clean in ["CPIN", "JPFA", "MAIN", "ICBP", "INDF", "GGRM", "HMSP", "UNVR", "KLBF"]:
        return "Consumer Goods"
    if symbol_clean in ["BBCA", "BBRI", "BMRI", "BBNI", "BTPS", "BJBR", "BJTM", "MEGA"]:
        return "Financials"
    if symbol_clean in ["TLKM", "JSMR", "EXCL", "ISAT", "TOWR", "WIKA", "PTPP", "WSKT", "ADHI"]:
        return "Infrastructure"
    if symbol_clean in ["CTRA", "PWON", "SMRA", "BSDE", "LPKR", "PPRO", "ASRI", "BKSL"]:
        return "Property"
    if symbol_clean in ["MTDL", "ASGR", "MAPI", "MCAS", "MLPL"]:
        return "Technology"
    if symbol_clean in ["ACES", "RALS", "ERAA", "LPPF", "MAPI", "MAPB"]:
        return "Retail"

    return "Other"

# ===================== FUNGSI MARKET SENTIMENT =====================
def get_market_sentiment(global_indices: dict) -> dict:
    """
    Menghitung sentimen pasar berdasarkan indeks global.
    """
    sentiment_score = 0
    sentiment_details = []

    if global_indices.get("IHSG", {}).get("value", 0) > 0:
        ihsg_mom = global_indices["IHSG"].get("momentum", 0)
        if ihsg_mom > 0.5:
            sentiment_score += 1
            sentiment_details.append("IHSG: BULLISH")
        elif ihsg_mom < -0.5:
            sentiment_score -= 1
            sentiment_details.append("IHSG: BEARISH")
        else:
            sentiment_details.append("IHSG: NETRAL")

    us_score = 0
    for idx in ["DOWJONES", "S&P500", "NASDAQ"]:
        if global_indices.get(idx, {}).get("value", 0) > 0:
            mom = global_indices[idx].get("momentum", 0)
            if mom > 0.5:
                us_score += 1
            elif mom < -0.5:
                us_score -= 1

    if us_score > 1:
        sentiment_score += 1
        sentiment_details.append("US: BULLISH")
    elif us_score < -1:
        sentiment_score -= 1
        sentiment_details.append("US: BEARISH")
    else:
        sentiment_details.append("US: NETRAL")

    if global_indices.get("HANGSENG", {}).get("value", 0) > 0:
        hs_mom = global_indices["HANGSENG"].get("momentum", 0)
        if hs_mom > 0.5:
            sentiment_score += 1
            sentiment_details.append("HANGSENG: BULLISH")
        elif hs_mom < -0.5:
            sentiment_score -= 1
            sentiment_details.append("HANGSENG: BEARISH")
        else:
            sentiment_details.append("HANGSENG: NETRAL")

    if global_indices.get("USDIDR", {}).get("value", 0) > 0:
        usd_mom = global_indices["USDIDR"].get("momentum", 0)
        if usd_mom < -0.5:
            sentiment_score += 1
            sentiment_details.append("USD/IDR: RUPIAH MENGUAT")
        elif usd_mom > 0.5:
            sentiment_score -= 1
            sentiment_details.append("USD/IDR: RUPIAH MELEMAH")
        else:
            sentiment_details.append("USD/IDR: NETRAL")

    if sentiment_score >= 2:
        market_condition = "🟢 BULLISH (Risk On)"
        threshold_adjustment = -0.05
        recommendation = "Kondisi pasar mendukung untuk beli"
    elif sentiment_score <= -2:
        market_condition = "🔴 BEARISH (Risk Off)"
        threshold_adjustment = +0.05
        recommendation = "Hati-hati, pasar sedang bearish"
    else:
        market_condition = "🟡 NETRAL (Sideways)"
        threshold_adjustment = 0
        recommendation = "Pasar sideways, selektif dalam memilih saham"

    # Tambahkan volatilitas untuk regime detection
    volatility_regime = "NORMAL"
    if 'IHSG' in global_indices and len(global_indices['IHSG'].get('returns', [])) > 20:
        returns = global_indices['IHSG']['returns']
        current_vol = returns.std() * np.sqrt(252)
        normal_vol = 0.15  # Asumsi volatilitas normal IHSG 15%

        if current_vol > normal_vol * 1.5:
            volatility_regime = "HIGH"
        elif current_vol < normal_vol * 0.7:
            volatility_regime = "LOW"

    return {
        "score": sentiment_score,
        "condition": market_condition,
        "volatility_regime": volatility_regime,
        "details": sentiment_details,
        "threshold_adjustment": threshold_adjustment,
        "recommendation": recommendation
    }

# ===================== FUNGSI KORELASI =====================
def calculate_correlation_matrix(stocks_data: list) -> np.ndarray:
    """
    Menghitung matriks korelasi antar saham berdasarkan return historis.
    """
    n = len(stocks_data)
    if n < 2:
        return np.array([[1]])

    # Kumpulkan returns
    returns_dict = {}
    for stock in stocks_data:
        if 'returns' in stock and len(stock['returns']) > 20:
            returns_dict[stock['Symbol']] = stock['returns']

    if len(returns_dict) < 2:
        return np.eye(n)

    # Buat dataframe returns
    df_returns = pd.DataFrame(returns_dict)
    corr_matrix = df_returns.corr().values

    return corr_matrix

def calculate_portfolio_risk(weights: np.ndarray, corr_matrix: np.ndarray, volatilities: np.ndarray) -> float:
    """
    Menghitung risiko portfolio berdasarkan korelasi dan volatilitas.
    """
    if len(weights) == 0:
        return 0

    weights = np.array(weights)
    weights = weights / weights.sum()  # Normalisasi

    # Portfolio variance = w^T * (corr * vol * vol^T) * w
    vol_matrix = np.outer(volatilities, volatilities)
    cov_matrix = corr_matrix * vol_matrix

    portfolio_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    portfolio_risk = np.sqrt(portfolio_var)

    return portfolio_risk

# ===================== THRESHOLD ADAPTIF =====================
def calculate_adaptive_threshold(
    prob_up: float,
    rsi: float,
    chart_bias: str,
    macd_hist: float,
    volume_ratio: float,
    dist_to_support: float,
    dist_to_resistance: float,
    global_indices: dict,
    sector: str,
    model_sharpe: float = 0,
    sentiment: dict = None
) -> float:
    """
    Menghitung threshold adaptif - dioptimasi untuk Sharpe.
    """
    # Base threshold lebih rendah (lebih agresif)
    base_threshold = 0.40
    adjustment = 0.0

    # ===== 1. TEKNIKAL ADJUSTMENT =====
    # RSI - lebih sensitif
    if rsi < 30:
        adjustment -= 0.15
    elif rsi < 40:
        adjustment -= 0.10
    elif rsi > 70:
        adjustment += 0.15
    elif rsi > 60:
        adjustment += 0.08

    # Chart bias
    if chart_bias == "BULLISH":
        adjustment -= 0.06
    elif chart_bias == "BEARISH":
        adjustment += 0.06

    # MACD
    if macd_hist > 0:
        adjustment -= 0.05
    else:
        adjustment += 0.05

    # Volume
    if volume_ratio > 1.5:
        adjustment -= 0.06
    elif volume_ratio < 0.5:
        adjustment += 0.06

    # Support/Resistance
    if dist_to_support < 0.01:
        adjustment -= 0.10
    elif dist_to_support < 0.02:
        adjustment -= 0.05

    if dist_to_resistance < 0.005:
        adjustment -= 0.12
    elif dist_to_resistance < 0.01:
        adjustment -= 0.06

    # ===== 2. SHARPE ADJUSTMENT =====
    if model_sharpe > 1.5:
        adjustment -= 0.08
    elif model_sharpe > 1.0:
        adjustment -= 0.05
    elif model_sharpe > 0.5:
        adjustment -= 0.02
    elif model_sharpe < 0:
        adjustment += 0.05

    # ===== 3. GLOBAL INDICES ADJUSTMENT =====
    if global_indices.get("IHSG", {}).get("value", 0) > 0:
        ihsg_momentum = global_indices["IHSG"].get("momentum", 0)
        if ihsg_momentum > 1.0:
            adjustment -= 0.08
        elif ihsg_momentum > 0.5:
            adjustment -= 0.04
        elif ihsg_momentum < -1.0:
            adjustment += 0.08
        elif ihsg_momentum < -0.5:
            adjustment += 0.04

    # ===== 4. SENTIMEN PASAR =====
    if sentiment:
        adjustment += sentiment.get("threshold_adjustment", 0)

    # Batasi adjustment
    adjustment = max(-0.40, min(0.40, adjustment))

    # Threshold final
    final_threshold = base_threshold + adjustment
    final_threshold = max(0.20, min(0.65, final_threshold))

    return final_threshold

# ===================== MODEL CANDIDATES =====================
def get_model_candidates():
    candidates = {
        "RandomForest": RandomForestClassifier(
            n_estimators=150, max_depth=5, min_samples_leaf=25,
            class_weight="balanced", random_state=SEED, n_jobs=-1
        ),
        "HistGradientBoosting": HistGradientBoostingClassifier(
            learning_rate=0.05, max_depth=3, max_iter=200,
            min_samples_leaf=25, random_state=SEED
        ),
        "XGBoost": xgb.XGBClassifier(
            n_estimators=150, learning_rate=0.05, max_depth=3,
            subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
            objective="multi:softprob", random_state=SEED, n_jobs=-1
        ),
    }
    return candidates

# ===================== WALK-FORWARD VALIDATION =====================
class WalkForwardValidator:
    """
    Walk-forward validation untuk mengukur Sharpe ratio.
    """
    def __init__(self, n_splits=5, train_pct=0.7, horizon=TARGET_HORIZON_BARS):
        self.n_splits = n_splits
        self.train_pct = train_pct
        self.horizon = horizon
        self.results = {}

    def validate(self, model, X, y, prices):
        """
        Validasi model dengan walk-forward.
        Returns: dict dengan Sharpe ratio dan metrik lainnya.
        """
        X_arr = np.asarray(X)
        y_arr = np.asarray(y)
        prices_arr = np.asarray(prices)
        n = len(y_arr)

        sharpe_ratios = []
        information_coefficients = []
        accuracies = []

        for i in range(self.n_splits):
            # Split data
            train_end = int(n * (self.train_pct + i * (1 - self.train_pct) / self.n_splits))
            train_start = max(0, train_end - int(n * self.train_pct))

            if train_end >= n - self.horizon - 10:
                break

            train_idx = np.arange(train_start, train_end)
            test_idx = np.arange(train_end, min(train_end + 50, n - self.horizon))

            if len(test_idx) < 20:
                continue

            X_train, y_train = X_arr[train_idx], y_arr[train_idx]
            X_test = X_arr[test_idx]

            # Future prices (NOTE: Ini asumsi entry di close, akan diperbaiki di production)
            future_idx = test_idx + self.horizon
            current_prices = prices_arr[test_idx]
            future_prices = prices_arr[future_idx]

            # Validasi data
            valid_mask = ~(np.isnan(current_prices) | np.isnan(future_prices) |
                          (current_prices <= 0) | (future_prices <= 0))

            if np.sum(valid_mask) < 10:
                continue

            current_prices_valid = current_prices[valid_mask]
            future_prices_valid = future_prices[valid_mask]
            X_test_valid = X_test[valid_mask]

            # Hitung forward returns
            forward_returns = (future_prices_valid / current_prices_valid) - 1

            # Train model
            model_clone = copy.deepcopy(model)
            try:
                model_clone.fit(X_train, y_train)
            except:
                continue

            # Predict
            if hasattr(model_clone, 'predict_proba'):
                try:
                    proba = model_clone.predict_proba(X_test_valid)
                    if proba.shape[1] > 2:
                        p_up = proba[:, 2]  # Kelas naik
                    elif proba.shape[1] > 1:
                        p_up = proba[:, 1]
                    else:
                        p_up = proba[:, 0]
                except:
                    continue
            else:
                p_up = model_clone.predict(X_test_valid) / 2

            # Signal
            signal = np.where(p_up > 0.5, 1, -1)

            # Strategy returns
            strategy_returns = signal * forward_returns

            # Sharpe ratio (annualized)
            if np.std(strategy_returns) > 1e-6:
                sharpe = np.mean(strategy_returns) / np.std(strategy_returns) * np.sqrt(252 * 7)
                sharpe_ratios.append(sharpe)

            # Information Coefficient
            try:
                ic, _ = stats.spearmanr(p_up, forward_returns)
                if not np.isnan(ic):
                    information_coefficients.append(ic)
            except:
                pass

            # Accuracy
            pred_direction = np.sign(signal)
            actual_direction = np.sign(forward_returns)
            acc = np.mean(pred_direction == actual_direction)
            accuracies.append(acc)

        return {
            'sharpe': np.mean(sharpe_ratios) if sharpe_ratios else 0,
            'ic': np.mean(information_coefficients) if information_coefficients else 0,
            'accuracy': np.mean(accuracies) if accuracies else 0,
            'n_folds': len(sharpe_ratios)
        }

# ===================== MODEL EVALUATOR =====================
class ModelEvaluator:
    def __init__(self, n_splits=N_SPLITS):
        self.n_splits = n_splits
        self.results = {}
        self.best_model = None
        self.best_model_name = None
        self.best_sharpe = -999
        self.best_sharpe_value = 0

    def evaluate(self, models, X, y, prices):
        """Evaluasi model berdasarkan SHARPE ratio (bukan accuracy)."""

        for name, model in models.items():
            print(f"\n📊 Validating {name}...")

            wfv = WalkForwardValidator(n_splits=self.n_splits)
            results = wfv.validate(model, X, y, prices)

            sharpe = results['sharpe']
            ic = results['ic']
            acc = results['accuracy']

            self.results[name] = {
                'sharpe': sharpe,
                'ic': ic,
                'accuracy': acc,
                'n_folds': results['n_folds']
            }

            if sharpe > self.best_sharpe:
                self.best_sharpe = sharpe
                self.best_sharpe_value = sharpe
                self.best_model_name = name
                self.best_model = model

            print(f"  Sharpe: {sharpe:.3f} | IC: {ic:.3f} | Acc: {acc:.2%} | Folds: {results['n_folds']}")

        return self

    def print_results(self):
        """Print perbandingan model berdasarkan Sharpe."""
        print("\n" + "=" * 100)
        print("📊 MODEL COMPARISON - WALK-FORWARD VALIDATION")
        print("=" * 100)

        table_data = []
        for name, res in self.results.items():
            table_data.append([
                name,
                f"{res['sharpe']:.3f}",
                f"{res['ic']:.3f}",
                f"{res['accuracy']:.2%}",
                res['n_folds']
            ])

        table_data.sort(key=lambda x: float(x[1]), reverse=True)
        headers = ["Model", "Sharpe", "Info Coef", "Accuracy", "Folds"]
        print(tabulate(table_data, headers=headers, tablefmt="grid"))

        if self.best_model_name:
            print(f"\n🏆 Best Model: {self.best_model_name} (Sharpe: {self.best_sharpe:.3f})")
            if self.best_sharpe > 1.0:
                print("✅ Model SANGAT VALID untuk trading")
            elif self.best_sharpe > 0.5:
                print("⚠️ Model CUKUP VALID")
            elif self.best_sharpe > 0:
                print("📊 Model VALID (positif)")
            else:
                print("❌ Model TIDAK VALID")

    def get_best_model_trained(self, X, y):
        if self.best_model_name is None:
            return None

        best_model_clone = copy.deepcopy(self.best_model)
        best_model_clone.fit(X, y)
        return best_model_clone

# ===================== ANALISIS CHART =====================
def analyze_chart_bias(df: pd.DataFrame) -> tuple[str, float]:
    try:
        close = df["Close"]
        volume = df["Volume"]

        ma20 = close.rolling(20).mean().iloc[-1]
        ma50 = close.rolling(50).mean().iloc[-1]
        ma_trend = 1 if ma20 > ma50 else -1 if ma20 < ma50 else 0

        macd = ta.trend.MACD(close)
        macd_line = macd.macd().iloc[-1]
        signal_line = macd.macd_signal().iloc[-1]
        macd_trend = 1 if macd_line > signal_line else -1 if macd_line < signal_line else 0

        rsi = ta.momentum.RSIIndicator(close).rsi().iloc[-1]
        rsi_trend = 1 if rsi > 55 else -1 if rsi < 45 else 0

        vol_ma20 = volume.rolling(20).mean().iloc[-1]
        vol_ratio = volume.iloc[-1] / vol_ma20 if vol_ma20 > 0 else 1
        volume_trend = 1 if vol_ratio > 1.2 else -1 if vol_ratio < 0.8 else 0

        weights = {'ma': 0.3, 'macd': 0.25, 'rsi': 0.25, 'volume': 0.2}
        score = (ma_trend * weights['ma'] + macd_trend * weights['macd'] +
                rsi_trend * weights['rsi'] + volume_trend * weights['volume'])

        strength = min(1, abs(score) * 2)

        if score > 0.15:
            bias = "BULLISH"
        elif score < -0.15:
            bias = "BEARISH"
        else:
            bias = "NEUTRAL"

        return bias, strength

    except Exception:
        return "NEUTRAL", 0.3

# ===================== HISTORICAL SUPPORT/RESISTANCE =====================
def find_historical_support_resistance(df: pd.DataFrame, lookback: int = SR_LOOKBACK) -> tuple[list, list]:
    if len(df) < lookback:
        lookback = len(df)

    high = df["High"].values[-lookback:]
    low = df["Low"].values[-lookback:]

    height_threshold = np.percentile(high, 70)
    peaks, _ = find_peaks(high, prominence=np.std(high) * SR_PROMINENCE,
                          height=height_threshold, distance=5)

    low_inverted = -low
    valleys, _ = find_peaks(low_inverted, prominence=np.std(low) * SR_PROMINENCE,
                           height=-np.percentile(low, 30), distance=5)

    resistance_levels = []
    for peak_idx in peaks:
        price = high[peak_idx]
        weight = 1.0 if peak_idx > len(high) - 50 else 0.7
        resistance_levels.append((price, weight))

    support_levels = []
    for valley_idx in valleys:
        price = low[valley_idx]
        weight = 1.0 if valley_idx > len(low) - 50 else 0.7
        support_levels.append((price, weight))

    resistance_levels.sort(key=lambda x: x[0])
    support_levels.sort(key=lambda x: x[0])

    return support_levels, resistance_levels

def get_nearest_support_resistance(price: float, support_levels: list, resistance_levels: list) -> tuple[float, float, float, float]:
    nearest_support = None
    nearest_support_weight = 0
    min_support_dist = float('inf')

    for s_price, weight in support_levels:
        if s_price < price:
            dist = price - s_price
            if dist < min_support_dist:
                min_support_dist = dist
                nearest_support = s_price
                nearest_support_weight = weight

    nearest_resistance = None
    nearest_resistance_weight = 0
    min_resistance_dist = float('inf')

    for r_price, weight in resistance_levels:
        if r_price > price:
            dist = r_price - price
            if dist < min_resistance_dist:
                min_resistance_dist = dist
                nearest_resistance = r_price
                nearest_resistance_weight = weight

    if nearest_support is None:
        nearest_support = price * 0.95
        nearest_support_weight = 0.5

    if nearest_resistance is None:
        nearest_resistance = price * 1.05
        nearest_resistance_weight = 0.5

    return nearest_support, nearest_support_weight, nearest_resistance, nearest_resistance_weight

# ===================== INTRADAY RISK MANAGEMENT (DENGAN FEE) =====================
def calculate_intraday_tp_sl(entry_price, atr, action, support=None, resistance=None,
                             support_weight=0.5, resistance_weight=0.5):
    if action is None:
        return None, None, 0

    atr = max(atr, 1) if atr is not None else 1

    multipliers = {
        'NORMAL BUY': {'sl': 1.5, 'tp': 2.0},
        'BUY ON PULLBACK': {'sl': 1.2, 'tp': 2.5},
        'BREAKOUT BUY': {'sl': 1.0, 'tp': 3.0},
    }.get(action, {'sl': 1.5, 'tp': 2.0})

    sl_price = entry_price - atr * multipliers['sl']
    tp_price = entry_price + atr * multipliers['tp']

    if action == "BREAKOUT BUY":
        if resistance and resistance > entry_price:
            tp_price = max(tp_price, resistance + atr * 1.5)
            sl_price = max(sl_price, resistance - atr * 0.5)

    elif action == "BUY ON PULLBACK":
        if resistance and resistance > entry_price:
            tp_price = min(tp_price, resistance - atr * 0.3)
        if support and support < entry_price:
            sl_price = min(sl_price, support - atr * 0.5)

    else:
        if resistance and resistance > entry_price:
            tp_price = min(tp_price, resistance - atr * 0.2)
        if support and support < entry_price:
            sl_price = max(sl_price, support - atr * 0.3)

    # Hitung risk/reward SETELAH FEE
    risk = entry_price - sl_price + (entry_price * FEE_BUY) + (sl_price * FEE_SELL)
    reward = tp_price - entry_price - (entry_price * FEE_BUY) - (tp_price * FEE_SELL)
    rr_ratio = reward / risk if risk > 0 else 0

    return round(sl_price, 0), round(tp_price, 0), round(rr_ratio, 2)

# ===================== CALIBRATED EXPECTED RETURN (DENGAN FEE) =====================
def calculate_calibrated_expected_return(prob_up, prob_down, tp_price, sl_price, entry_price, action):
    if action is None or tp_price is None or sl_price is None:
        return {'pct': 0, 'rp_per_share': 0, 'calibrated_prob_up': prob_up}

    # Kalibrasi probabilitas (lebih konservatif)
    calibrated_prob_up = 0.5 + (prob_up - 0.5) * 0.7
    calibrated_prob_up = max(0.3, min(0.7, calibrated_prob_up))

    # Hitung return setelah fee
    gain = (tp_price - entry_price) / entry_price - TOTAL_FEE
    loss = (sl_price - entry_price) / entry_price - TOTAL_FEE

    expected_return_pct = (calibrated_prob_up * gain) + ((1 - calibrated_prob_up) * loss)
    expected_return_rp = expected_return_pct * entry_price

    return {
        'pct': round(expected_return_pct * 100, 2),
        'rp_per_share': round(expected_return_rp, 0),
        'calibrated_prob_up': round(calibrated_prob_up, 3)
    }

# ===================== FUNGSI PROBABILITAS REALISTIS =====================
def calculate_realistic_probability(df_feat: pd.DataFrame, model_proba: tuple = None) -> tuple:
    prob_up = 0.5
    prob_down = 0.3
    prob_side = 0.2

    scores = []
    weights = []

    if model_proba is not None:
        if len(model_proba) == 3:
            ml_up = model_proba[2]
            ml_down = model_proba[0]
        else:
            ml_up = model_proba[1] if len(model_proba) > 1 else 0.5
            ml_down = model_proba[0] if len(model_proba) > 0 else 0.3

        scores.append(ml_up - ml_down)
        weights.append(0.4)

    if "RSI" in df_feat.columns:
        rsi = float(df_feat["RSI"].iloc[-1])
        if not np.isnan(rsi):
            if rsi < 30:
                rsi_score = 0.3
            elif rsi < 40:
                rsi_score = 0.2
            elif rsi < 50:
                rsi_score = 0.1
            elif rsi < 60:
                rsi_score = -0.1
            elif rsi < 70:
                rsi_score = -0.2
            else:
                rsi_score = -0.3
            scores.append(rsi_score)
            weights.append(0.2)

    if "MACD_Hist" in df_feat.columns:
        macd_hist = float(df_feat["MACD_Hist"].iloc[-1])
        if not np.isnan(macd_hist):
            macd_score = np.tanh(macd_hist * 10) * 0.3
            scores.append(macd_score)
            weights.append(0.15)

    if "Volume_Ratio" in df_feat.columns:
        vol_ratio = float(df_feat["Volume_Ratio"].iloc[-1])
        if not np.isnan(vol_ratio) and vol_ratio > 0:
            vol_score = (np.log(vol_ratio) * 0.1)
            vol_score = max(-0.2, min(0.2, vol_score))
            scores.append(vol_score)
            weights.append(0.1)

    # Global momentum
    global_score = 0
    global_count = 0
    for col in df_feat.columns:
        if "_Momentum" in col:
            val = float(df_feat[col].iloc[-1])
            if not np.isnan(val):
                global_score += val * 0.05
                global_count += 1

    if global_count > 0:
        global_score = global_score / global_count
        global_score = max(-0.2, min(0.2, global_score))
        scores.append(global_score)
        weights.append(0.05)

    if scores and weights:
        weights = np.array(weights) / sum(weights)
        total_score = sum(s * w for s, w in zip(scores, weights))

        # ===== UNCERTAINTY PENALTY (ANTI-OVERFITTING) =====
        model_confidence = abs(prob_up - 0.5) * 2   # 0 → tidak yakin, 1 → sangat yakin

        # shrink toward randomness
        shrink_factor = 0.6 + 0.4 * model_confidence

        prob_up = 0.5 + (prob_up - 0.5) * shrink_factor

        # batas akhir
        prob_up = max(0.35, min(0.65, prob_up))

        prob_down = (1 - prob_up) * 0.6
        prob_side = (1 - prob_up) * 0.4

    return prob_up, prob_down, prob_side

# ===================== FEATURE ENGINEERING =====================
def calculate_features_optimized(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Hitung fitur teknikal dengan caching.
    """
    cache_key = hashlib.md5(f"{df_raw.shape[0]}_{df_raw['Close'].iloc[-1]}_{df_raw.index[-1]}".encode()).hexdigest()

    if cache_key in _feature_cache:
        return _feature_cache[cache_key].copy()

    out = df_raw.copy()

    close = out["Close"]
    high = out["High"]
    low = out["Low"]
    volume = out["Volume"]

    # Returns
    out["Return_1"] = close.pct_change()
    out["Return_5"] = close.pct_change(5)
    out["Return_10"] = close.pct_change(10)
    out["Return_20"] = close.pct_change(20)

    # Volatility
    out["Volatility_10"] = out["Return_1"].rolling(10).std()
    out["Volatility_20"] = out["Return_1"].rolling(20).std()

    # Volume
    out["Volume_MA20"] = volume.rolling(20).mean()
    out["Volume_Ratio"] = volume / (out["Volume_MA20"] + 1)

    # Moving averages
    out["MA20"] = close.rolling(20).mean()
    out["MA50"] = close.rolling(50).mean()
    out["MA_Spread"] = (out["MA20"] - out["MA50"]) / close

    # Momentum
    out["RSI"] = ta.momentum.RSIIndicator(close, window=14).rsi()
    out["MACD"] = ta.trend.MACD(close).macd()
    out["MACD_Signal"] = ta.trend.MACD(close).macd_signal()
    out["MACD_Hist"] = out["MACD"] - out["MACD_Signal"]

    # ATR
    out["ATR"] = ta.volatility.AverageTrueRange(high, low, close, window=14).average_true_range()

    # Support & Resistance
    out["Support_20"] = low.rolling(20).min()
    out["Resistance_20"] = high.rolling(20).max()
    out["Dist_to_Support"] = (close - out["Support_20"]) / close
    out["Dist_to_Resistance"] = (out["Resistance_20"] - close) / close

    # Price position
    out["Price_Position"] = (close - out["Support_20"]) / (out["Resistance_20"] - out["Support_20"] + 0.001)

    # Tambahkan fitur global
    out = add_global_indices_features_optimized(out)

    # Labels - PAKAI VOLATILITY-NORMALIZED RETURN
    out["Future_Return"] = (out["Close"].shift(-TARGET_HORIZON_BARS) / out["Close"]) - 1

    # Normalisasi dengan volatility
    out["Volatility_Norm"] = out["Return_1"].rolling(20).std()
    out["Return_Vol_Ratio"] = out["Future_Return"] / (out["Volatility_Norm"] + 0.001)

    # Label berdasarkan return/volatility ratio
    conditions = [
        out["Return_Vol_Ratio"] < -0.3,
        (out["Return_Vol_Ratio"] >= -0.3) & (out["Return_Vol_Ratio"] <= 0.3),
        out["Return_Vol_Ratio"] > 0.3
    ]
    out["Target"] = np.select(conditions, [0, 1, 2], default=1).astype(int)

    out = replace_inf_with_nan(out)

    _feature_cache[cache_key] = out.copy()

    return out

# ===================== DETERMINE ACTION =====================
def determine_action_based_on_sr(
    last_price: float,
    support: float,
    resistance: float,
    prob_up: float,
    chart_bias: str,
    rsi: float,
    macd_hist: float,
    volume_ratio: float,
    global_indices: dict,
    sector: str,
    model_sharpe: float = 0,
    sentiment: dict = None
) -> tuple[str | None, float]:
    """
    Menentukan action berdasarkan support/resistance.
    """

    dist_to_support = (last_price - support) / last_price if support > 0 else 1
    dist_to_resistance = (resistance - last_price) / last_price if resistance > 0 else 1

    # Hitung threshold adaptif
    threshold = calculate_adaptive_threshold(
        prob_up, rsi, chart_bias, macd_hist, volume_ratio,
        dist_to_support, dist_to_resistance,
        global_indices, sector, model_sharpe,
        sentiment
    )

    # Jika probabilitas rendah tapi kondisi teknikal sangat bagus
    if prob_up < threshold:
        if (rsi < 30 and dist_to_support < 0.02) or (dist_to_support < 0.01 and volume_ratio > 1.5):
            return "BUY ON PULLBACK (FALLBACK)", threshold
        return None, threshold

    # Breakout
    if last_price > resistance * 1.001:
        return "BREAKOUT BUY", threshold

    # Pullback
    if dist_to_support < 0.03:
        return "BUY ON PULLBACK", threshold

    # Normal buy
    if prob_up > 0.40:
        return "NORMAL BUY", threshold

    return None, threshold

# ===================== CONFIDENCE INTERVAL =====================
def calculate_confidence_interval(current_price: float, volatility: float, prob_up: float, days: int = 2) -> dict:
    """
    Menghitung confidence interval (bukan path prediction).
    """
    period_vol = volatility * np.sqrt(days * 7)  # 7 jam per hari
    expected_return = (prob_up - 0.5) * period_vol * 1.5

    ci_68_low = current_price * (1 + expected_return - period_vol)
    ci_68_high = current_price * (1 + expected_return + period_vol)

    return {
        'expected_price': round(current_price * (1 + expected_return), 0),
        'expected_return': f"{expected_return*100:.2f}%",
        'ci_68': (round(ci_68_low, 0), round(ci_68_high, 0)),
        'prob_up': prob_up
    }

# ===================== PRODUCTION-READY COMPONENTS =====================

class CapitalTierDetector:
    """
    Mendeteksi tier modal untuk penyesuaian strategi
    """
    def __init__(self, modal):
        self.modal = modal
        self.tier = self._determine_tier(modal)
        self.max_position_pct = self._get_max_position_pct()
        self.max_sector_pct = self._get_max_sector_pct()
        self.liquidity_requirement = self._get_liquidity_requirement()

    def _determine_tier(self, modal):
        if modal < 200000:
            return 'MICRO'
        elif modal < 1000000:
            return 'SMALL'
        elif modal < 3000000:
            return 'MEDIUM'
        else:
            return 'LARGE'

    def _get_max_position_pct(self):
        tiers = {
            'MICRO': 0.50,
            'SMALL': 0.30,
            'MEDIUM': 0.20,
            'LARGE': 0.15
        }
        return tiers.get(self.tier, 0.20)

    def _get_max_sector_pct(self):
        tiers = {
            'MICRO': 0.60,
            'SMALL': 0.50,
            'MEDIUM': 0.40,
            'LARGE': 0.30
        }
        return tiers.get(self.tier, 0.40)

    def _get_liquidity_requirement(self):
        tiers = {
            'MICRO': 0.01,   # 1% dari volume harian
            'SMALL': 0.03,   # 3% dari volume harian
            'MEDIUM': 0.05,  # 5% dari volume harian
            'LARGE': 0.08    # 8% dari volume harian
        }
        return tiers.get(self.tier, 0.03)

    def get_position_size(self, stock_price, daily_volume_rp, base_lot=1):
        """
        Hitung ukuran posisi yang aman berdasarkan tier
        """
        # Batasan modal
        max_by_modal = int((self.modal * self.max_position_pct) / (stock_price * 100))

        # Batasan likuiditas
        max_by_liquidity = int((daily_volume_rp * self.liquidity_requirement) / (stock_price * 100))

        # Ambil yang lebih kecil (konservatif)
        recommended_lot = min(max_by_modal, max_by_liquidity, base_lot)

        return max(recommended_lot, 1)


class ExecutionAdvisor:
    """
    Simulasi dan advis eksekusi manusia
    """
    def __init__(self):
        self.human_delay_distribution = {
            'cepat': (1, 3),      # 1-3 menit
            'normal': (3, 7),     # 3-7 menit
            'lambat': (7, 15),    # 7-15 menit
            'very_late': (15, 30) # 15-30 menit
        }

        self.fill_probability = {
            'limit_better': 0.3,   # Limit lebih baik dari market
            'limit_equal': 0.7,    # Limit sama dengan market
            'market': 0.95         # Market order (dengan slippage)
        }

    def estimate_execution_quality(self, signal_price, volume_lot, daily_volume_rp, human_type='normal'):
        """
        Estimasi kualitas eksekusi
        Returns: (expected_price, fill_probability, execution_advice)
        """
        # 1. Delay
        delay_min, delay_max = self.human_delay_distribution[human_type]
        expected_delay = (delay_min + delay_max) / 2

        # 2. Fill probability
        volume_rp = volume_lot * 100 * signal_price
        volume_ratio = volume_rp / daily_volume_rp if daily_volume_rp > 0 else 0.1

        if volume_ratio < 0.01:
            fill_prob = 0.95
        elif volume_ratio < 0.03:
            fill_prob = 0.85
        elif volume_ratio < 0.05:
            fill_prob = 0.70
        else:
            fill_prob = 0.50

        # 3. Slippage estimate
        if volume_ratio < 0.01:
            slippage = 0.001  # 0.1%
        elif volume_ratio < 0.03:
            slippage = 0.003  # 0.3%
        elif volume_ratio < 0.05:
            slippage = 0.005  # 0.5%
        else:
            slippage = 0.01   # 1%

        # 4. Expected price after delay and slippage
        expected_price = signal_price * (1 + slippage)

        # 5. Execution advice
        if fill_prob < 0.6:
            advice = "⚠️ Risiko eksekusi tinggi, pertimbangkan market order"
        elif volume_ratio > 0.05:
            advice = "⚠️ Volume besar, eksekusi bertahap"
        elif expected_delay > 10:
            advice = "⏰ Delay panjang, gunakan limit order lebih agresif"
        else:
            advice = "✅ Eksekusi normal"

        return expected_price, fill_prob, advice

    def simulate_execution(self, signal_price, volume_lot, daily_volume_rp, volatility, human_type='normal'):
        """
        Simulasi eksekusi realistis
        Returns: (executed_price, filled, partial_fill)
        """
        # Random delay
        delay_min, delay_max = self.human_delay_distribution[human_type]
        delay = np.random.uniform(delay_min, delay_max)

        # Price movement during delay
        price_change = np.random.normal(0, volatility * np.sqrt(delay / (60*24)))

        # Fill probability
        volume_rp = volume_lot * 100 * signal_price
        volume_ratio = volume_rp / daily_volume_rp if daily_volume_rp > 0 else 0.1

        if volume_ratio < 0.01:
            fill_prob = 0.95
            partial_prob = 0.05
        elif volume_ratio < 0.03:
            fill_prob = 0.85
            partial_prob = 0.10
        elif volume_ratio < 0.05:
            fill_prob = 0.70
            partial_prob = 0.20
        else:
            fill_prob = 0.50
            partial_prob = 0.30

        rand = np.random.random()
        if rand < fill_prob:
            # Full fill
            executed_price = signal_price * (1 + price_change) * (1 + 0.002)  # +0.2% slippage
            return round(executed_price), True, False
        elif rand < fill_prob + partial_prob:
            # Partial fill (50% lot)
            executed_price = signal_price * (1 + price_change) * (1 + 0.003)  # +0.3% slippage
            return round(executed_price), True, True
        else:
            # No fill
            return None, False, False


class TradeHistoryTracker:
    """
    Melacak history trading untuk risk management
    """
    def __init__(self, max_history=100):
        self.trades = deque(maxlen=max_history)
        self.daily_pnl = deque(maxlen=30)
        self.consecutive_losses = 0
        self.last_update_date = None

    def add_trade(self, trade_data):
        """Tambahkan trade ke history"""
        with trade_history_lock:
            trade_data['timestamp'] = datetime.now()
            self.trades.append(trade_data)

            # Update consecutive losses
            if trade_data.get('pnl', 0) < 0:
                self.consecutive_losses += 1
            else:
                self.consecutive_losses = 0

            # Update daily PnL
            today = datetime.now().date()
            if self.last_update_date != today:
                self.daily_pnl.append(0)
                self.last_update_date = today

            if self.daily_pnl:
                self.daily_pnl[-1] += trade_data.get('pnl', 0)

    def get_metrics(self):
        """Dapatkan metrik terkini"""
        metrics = {}

        if len(self.trades) >= 10:
            recent_trades = list(self.trades)[-20:]
            wins = sum(1 for t in recent_trades if t.get('pnl', 0) > 0)
            metrics['win_rate_20'] = wins / len(recent_trades) if recent_trades else 0

            # Sharpe ratio dari trades terakhir
            returns = [t.get('return_pct', 0) for t in recent_trades if t.get('return_pct')]
            if len(returns) > 5 and np.std(returns) > 0:
                metrics['rolling_sharpe'] = (np.mean(returns) / np.std(returns)) * np.sqrt(252)
            else:
                metrics['rolling_sharpe'] = 0

        metrics['consecutive_losses'] = self.consecutive_losses
        metrics['total_trades'] = len(self.trades)
        metrics['daily_pnl'] = sum(self.daily_pnl) if self.daily_pnl else 0

        return metrics


class AutomaticRiskReduction:
    """
    Mengurangi risiko secara otomatis berdasarkan kondisi
    """
    def __init__(self, trade_history):
        self.trade_history = trade_history
        self.position_multiplier = 1.0
        self.min_prob_threshold = 0.50
        self.min_rr_threshold = 1.5

    def update(self):
        """Update multiplier berdasarkan kondisi terkini"""
        metrics = self.trade_history.get_metrics()

        # Base multiplier
        multiplier = 1.0

        # Consecutive losses penalty
        cons_loss = metrics.get('consecutive_losses', 0)
        if cons_loss >= 5:
            multiplier = 0.0
        elif cons_loss >= 3:
            multiplier *= 0.5

        # Win rate penalty
        win_rate = metrics.get('win_rate_20', 0.5)
        if win_rate < 0.35:
            multiplier *= 0.5
        elif win_rate < 0.40:
            multiplier *= 0.7

        # Daily loss penalty
        daily_pnl = metrics.get('daily_pnl', 0)
        if daily_pnl < -0.03:  # -3% dalam sehari
            multiplier *= 0.5

        self.position_multiplier = max(0, multiplier)

        # Adjust thresholds
        if self.position_multiplier < 0.3:
            self.min_prob_threshold = 0.60
            self.min_rr_threshold = 2.0
        elif self.position_multiplier < 0.7:
            self.min_prob_threshold = 0.55
            self.min_rr_threshold = 1.8
        else:
            self.min_prob_threshold = 0.50
            self.min_rr_threshold = 1.5

    def should_trade(self, signal):
        """Cek apakah sinyal boleh dieksekusi"""
        if self.position_multiplier == 0:
            return False, "ARR: Stop trading - risk reduction active"

        if signal['Prob_Naik'] < self.min_prob_threshold:
            return False, f"ARR: Prob {signal['Prob_Naik']:.0%} < {self.min_prob_threshold:.0%}"

        if signal['RR_Ratio'] < self.min_rr_threshold:
            return False, f"ARR: RR {signal['RR_Ratio']} < {self.min_rr_threshold}"

        return True, "OK"

    def get_position_size(self, base_size):
        """Dapatkan ukuran posisi setelah adjustment"""
        return int(base_size * self.position_multiplier)


class TradingCooldown:
    """
    Sistem pendinginan saat performa menurun
    """
    def __init__(self):
        self.cooldown_active = False
        self.cooldown_reason = ""
        self.cooldown_until = None
        self.cooldown_start = None

    def check(self, trade_history, current_time):
        """
        Cek apakah perlu cooldown
        Returns: (cooldown_active, reason, remaining_hours)
        """
        if self.cooldown_active:
            if current_time >= self.cooldown_until:
                self.cooldown_active = False
                self.cooldown_reason = ""
                return False, "", 0
            else:
                remaining = (self.cooldown_until - current_time).total_seconds() / 3600
                return True, self.cooldown_reason, remaining

        metrics = trade_history.get_metrics()

        # Kondisi cooldown
        # 1. 5 consecutive losses
        if metrics.get('consecutive_losses', 0) >= 5:
            self._activate("5 consecutive losses", current_time, hours=24)
            return True, self.cooldown_reason, 24

        # 2. Daily loss > 5%
        if metrics.get('daily_pnl', 0) < -0.05:
            self._activate("Daily loss > 5%", current_time, hours=24)
            return True, self.cooldown_reason, 24

        # 3. Win rate < 30% in last 20 trades
        win_rate = metrics.get('win_rate_20', 0.5)
        if win_rate < 0.30 and metrics.get('total_trades', 0) >= 20:
            self._activate(f"Win rate {win_rate:.0%} < 30%", current_time, hours=48)
            return True, self.cooldown_reason, 48

        return False, "", 0

    def _activate(self, reason, current_time, hours=24):
        self.cooldown_active = True
        self.cooldown_reason = reason
        self.cooldown_start = current_time
        self.cooldown_until = current_time + timedelta(hours=hours)

        # Log ke file
        with open('cooldown_log.txt', 'a') as f:
            f.write(f"{current_time}: COOLDOWN ACTIVATED - {reason} until {self.cooldown_until}\n")


class CapitalPreservationMode:
    """
    Mode preservasi modal saat kondisi ekstrem
    """
    def __init__(self, modal_awal):
        self.modal_awal = modal_awal
        self.modal_sekarang = modal_awal
        self.peak_modal = modal_awal
        self.mode = 'NORMAL'
        self.history = []

    def update(self, current_modal):
        """Update modal dan cek mode"""
        self.modal_sekarang = current_modal
        self.peak_modal = max(self.peak_modal, current_modal)

        # Hitung drawdown dari peak
        drawdown = (self.peak_modal - current_modal) / self.peak_modal

        # Tentukan mode
        if drawdown > 0.15:  # 15% dari peak
            self.mode = 'PRESERVATION'
        elif drawdown > 0.10:  # 10% dari peak
            self.mode = 'CAUTIOUS'
        else:
            self.mode = 'NORMAL'

        self.history.append({
            'time': datetime.now(),
            'modal': current_modal,
            'mode': self.mode,
            'drawdown': drawdown
        })

        return self.mode, drawdown

    def get_trading_rules(self):
        """Dapatkan aturan trading berdasarkan mode"""
        rules = {
            'NORMAL': {
                'max_position': 1.0,
                'min_prob': 0.50,
                'min_rr': 1.5,
                'max_trades_per_day': 5,
                'description': 'Trading normal'
            },
            'CAUTIOUS': {
                'max_position': 0.5,
                'min_prob': 0.55,
                'min_rr': 2.0,
                'max_trades_per_day': 3,
                'description': 'Mode hati-hati - kurangi exposure'
            },
            'PRESERVATION': {
                'max_position': 0.0,
                'min_prob': 0.60,
                'min_rr': 2.5,
                'max_trades_per_day': 0,
                'description': 'Mode preservasi - STOP TRADING'
            }
        }
        return rules.get(self.mode, rules['NORMAL'])

    def should_trade(self, signal):
        """Cek apakah sinyal boleh dieksekusi di mode ini"""
        rules = self.get_trading_rules()

        if rules['max_position'] == 0:
            return False, f"🚫 Capital preservation mode - {rules['description']}"

        if signal['Prob_Naik'] < rules['min_prob']:
            return False, f"🚫 Prob {signal['Prob_Naik']:.0%} < {rules['min_prob']:.0%} ({self.mode} mode)"

        if signal['RR_Ratio'] < rules['min_rr']:
            return False, f"🚫 RR {signal['RR_Ratio']} < {rules['min_rr']} ({self.mode} mode)"

        return True, f"✅ {self.mode} mode - trading allowed"


class KillSwitch:
    """
    Mekanisme penghentian darurat
    """
    def __init__(self, modal_awal):
        self.modal_awal = modal_awal
        self.kill_switch_active = False
        self.kill_reason = ""
        self.kill_time = None

        # Threshold
        self.max_absolute_loss = 0.20  # 20% dari modal awal
        self.max_drawdown = 0.25       # 25% dari peak
        self.max_consecutive_losses = 7
        self.min_win_rate_20 = 0.30    # 30% win rate dalam 20 trade

    def check(self, current_modal, trade_history):
        """
        Cek apakah kill switch perlu diaktifkan
        Returns: (should_kill, reason)
        """
        if self.kill_switch_active:
            return True, self.kill_reason

        metrics = trade_history.get_metrics()

        # 1. Absolute loss
        loss_pct = (self.modal_awal - current_modal) / self.modal_awal
        if loss_pct > self.max_absolute_loss:
            self._activate(f"Absolute loss {loss_pct:.1%} > {self.max_absolute_loss:.0%}")
            return True, self.kill_reason

        # 2. Drawdown from peak
        peak = self.modal_awal
        for h in getattr(trade_history, 'trades', []):
            if hasattr(h, 'get') and h.get('modal_after'):
                peak = max(peak, h['modal_after'])

        drawdown = (peak - current_modal) / peak
        if drawdown > self.max_drawdown:
            self._activate(f"Drawdown {drawdown:.1%} > {self.max_drawdown:.0%}")
            return True, self.kill_reason

        # 3. Consecutive losses
        cons_loss = metrics.get('consecutive_losses', 0)
        if cons_loss >= self.max_consecutive_losses:
            self._activate(f"{cons_loss} consecutive losses")
            return True, self.kill_reason

        # 4. Win rate in last 20 trades
        win_rate = metrics.get('win_rate_20', 0.5)
        if metrics.get('total_trades', 0) >= 20 and win_rate < self.min_win_rate_20:
            self._activate(f"Win rate {win_rate:.1%} < {self.min_win_rate_20:.0%}")
            return True, self.kill_reason

        return False, ""

    def _activate(self, reason):
        """Aktifkan kill switch"""
        self.kill_switch_active = True
        self.kill_reason = reason
        self.kill_time = datetime.now()

        # Log ke file
        with open('kill_switch_log.txt', 'a') as f:
            f.write(f"{self.kill_time}: 🔴 KILL SWITCH ACTIVATED - {reason}\n")

    def reset(self):
        """Reset kill switch (hanya untuk emergency)"""
        self.kill_switch_active = False
        self.kill_reason = ""
        self.kill_time = None


class ModelHealthMonitor:
    """
    Memantau kesehatan model secara real-time
    """
    def __init__(self):
        self.metrics = {
            'rolling_sharpe_20': deque(maxlen=20),
            'rolling_ic_20': deque(maxlen=20),
            'win_rate_20': deque(maxlen=20),
            'avg_rr_20': deque(maxlen=20),
            'execution_rate': deque(maxlen=20)
        }

        self.thresholds = {
            'healthy': {
                'rolling_sharpe_20': 0.5,
                'win_rate_20': 0.45,
                'avg_rr_20': 1.2,
                'execution_rate': 0.7
            },
            'warning': {
                'rolling_sharpe_20': 0.0,
                'win_rate_20': 0.40,
                'avg_rr_20': 1.0,
                'execution_rate': 0.6
            }
        }

        self.daily_checks = []

    def update(self, trade_result):
        """Update dengan hasil trade terbaru"""
        # Ini akan diisi dengan data real dari trading
        pass

    def get_status(self):
        """
        Dapatkan status model
        Returns: (status, reason)
        """
        if len(self.metrics['win_rate_20']) < 10:
            return 'INSUFFICIENT_DATA', "Data belum cukup"

        status = 'HEALTHY'
        reasons = []

        # Cek win rate
        if self.metrics['win_rate_20']:
            current_wr = np.mean(list(self.metrics['win_rate_20'])[-5:])
            if current_wr < self.thresholds['warning']['win_rate_20']:
                return 'CRITICAL', f"Win rate {current_wr:.1%}"
            elif current_wr < self.thresholds['healthy']['win_rate_20']:
                status = 'WARNING'
                reasons.append(f"WR={current_wr:.1%}")

        # Cek RR
        if self.metrics['avg_rr_20']:
            current_rr = np.mean(list(self.metrics['avg_rr_20'])[-5:])
            if current_rr < self.thresholds['warning']['avg_rr_20']:
                return 'CRITICAL', f"RR {current_rr:.1f}"
            elif current_rr < self.thresholds['healthy']['avg_rr_20']:
                status = 'WARNING'
                reasons.append(f"RR={current_rr:.1f}")

        # Cek execution rate
        if self.metrics['execution_rate']:
            current_er = np.mean(list(self.metrics['execution_rate'])[-5:])
            if current_er < self.thresholds['warning']['execution_rate']:
                return 'CRITICAL', f"Execution {current_er:.1%}"
            elif current_er < self.thresholds['healthy']['execution_rate']:
                status = 'WARNING'
                reasons.append(f"Exec={current_er:.1%}")

        if reasons:
            return status, ", ".join(reasons)
        else:
            return status, "All metrics healthy"


class DecisionIntelligence:
    """
    Layer insight untuk membantu keputusan manusia
    """
    def __init__(self, capital_tier, trade_history, market_sentiment):
        self.capital_tier = capital_tier
        self.trade_history = trade_history
        self.market_sentiment = market_sentiment

    def generate_insights(self, signal, portfolio, execution_advice=None):
        """
        Generate insight untuk setiap sinyal
        """
        insights = []

        # 1. Execution Confidence
        if execution_advice:
            insights.append(f"💡 Eksekusi: {execution_advice}")
        else:
            price = signal['Harga']
            if price < 200:
                insights.append("💡 Eksekusi: Harga rendah, spread bisa 1-2%, gunakan limit order")
            elif price > 500:
                insights.append("💡 Eksekusi: Likuiditas cukup, market order aman")

        # 2. Market Context
        if self.market_sentiment.get('condition') == '🔴 BEARISH (Risk Off)':
            insights.append("⚠️ Market Bearish: Sinyal BUY perlu konfirmasi tambahan")
        elif self.market_sentiment.get('condition') == '🟢 BULLISH (Risk On)':
            insights.append("✅ Market Bullish: Kondisi mendukung")

        # 3. Volatility Regime
        vol_regime = self.market_sentiment.get('volatility_regime', 'NORMAL')
        if vol_regime == 'HIGH':
            insights.append("🌪️ High Volatility: Stop loss diperlebar, probabilitas >55% disarankan")
        elif vol_regime == 'LOW':
            insights.append("🍃 Low Volatility: Target harga lebih sulit tercapai, pertimbangkan take profit lebih cepat")

        # 4. Timing Sensitivity
        if "BREAKOUT" in signal['Action']:
            insights.append("⏰ Timing: BREAKOUT sensitif waktu, eksekusi segera")
        elif "PULLBACK" in signal['Action'] and signal['RR_Ratio'] > 2.0:
            insights.append("⏰ Timing: PULLBACK dengan RR tinggi, tunggu konfirmasi di support")

        # 5. Correlation Warning
        same_sector = sum(1 for p in portfolio if p['Sector'] == signal['Sector'])
        if same_sector >= 2:
            insights.append(f"⚠️ Konsentrasi: Sudah {same_sector} saham di sektor {signal['Sector']}")

                # 6. Probabilistic Insight
        prob = signal['Prob_Naik']
        if prob > 0.65:
            insights.append(f"📊 Probabilitas {prob:.0%}: Sinyal kuat, eksekusi dengan market order")
        elif prob < 0.45:
            insights.append(f"📊 Probabilitas {prob:.0%}: Sinyal lemah, pertimbangkan limit order lebih rendah")

        # 7. Capital Tier Advice
        if self.capital_tier == 'MICRO':
            insights.append("💰 Modal mikro: Fokus pada 1-2 saham terbaik, hindari over-diversifikasi")
        elif self.capital_tier == 'LARGE':
            insights.append("💰 Modal besar: Prioritaskan saham likuid, hindari saham dengan volume < Rp 50jt/hari")

        # 8. Recent Performance Context
        metrics = self.trade_history.get_metrics()
        if metrics.get('consecutive_losses', 0) >= 2:
            insights.append(f"⚠️ Sedang dalam {metrics['consecutive_losses']} loss beruntun, pertimbangkan ukuran posisi lebih kecil")

        return insights

    def generate_market_summary(self):
        """Generate summary pasar"""
        summary = []
        summary.append(f"📊 Kondisi: {self.market_sentiment['condition']}")
        summary.append(f"📊 Volatilitas: {self.market_sentiment.get('volatility_regime', 'NORMAL')}")

        metrics = self.trade_history.get_metrics()
        if metrics:
            summary.append(f"📊 Win Rate (20): {metrics.get('win_rate_20', 0):.0%}")
            summary.append(f"📊 Consecutive Losses: {metrics.get('consecutive_losses', 0)}")

        return summary

    def generate_trading_tips(self):
        """Generate tips trading harian"""
        tips = []
        tips.append("• Eksekusi terbaik: 5-15 menit pertama setelah sinyal")
        tips.append("• Hindari entry 30 menit sebelum closing (sore)")

        if self.market_sentiment.get('volatility_regime') == 'HIGH':
            tips.append("• High vol: Pasang stop loss lebih lebar dari biasanya")

        if self.capital_tier == 'MICRO':
            tips.append("• Modal kecil: 1 lot cukup, jangan paksakan diversifikasi")

        return tips


class ProductionPortfolioAllocator:
    """
    Versi production dari CorrelationAwareAllocator dengan semua komponen risk
    """
    def __init__(self, modal, capital_tier_detector, risk_reduction, cooldown,
                 preservation, kill_switch, health_monitor, decision_intel):
        self.modal = modal
        self.capital_tier = capital_tier_detector
        self.risk_reduction = risk_reduction
        self.cooldown = cooldown
        self.preservation = preservation
        self.kill_switch = kill_switch
        self.health_monitor = health_monitor
        self.decision_intel = decision_intel
        self.remaining_modal = modal
        self.allocated = []
        self.insights = []

    def allocate(self, stocks, current_time):
        """
        Alokasi modal dengan semua lapisan proteksi
        """
        if not stocks:
            return [], 0, []

        # ===== 1. CHECK KILL SWITCH =====
        kill, kill_reason = self.kill_switch.check(self.remaining_modal, self.decision_intel.trade_history)
        if kill:
            print(f"\n🔴 KILL SWITCH ACTIVE: {kill_reason}")
            return [], 0, [f"🔴 KILL SWITCH: {kill_reason}"]

        # ===== 2. CHECK COOLDOWN =====
        cooldown_active, cooldown_reason, remaining_hours = self.cooldown.check(
            self.decision_intel.trade_history, current_time
        )
        if cooldown_active:
            print(f"\n⏸️ COOLDOWN ACTIVE: {cooldown_reason} ({remaining_hours:.1f} jam)")
            return [], 0, [f"⏸️ COOLDOWN: {cooldown_reason} - {remaining_hours:.1f} jam lagi"]

        # ===== 3. UPDATE CAPITAL PRESERVATION =====
        preservation_mode, drawdown = self.preservation.update(self.remaining_modal)
        preservation_rules = self.preservation.get_trading_rules()

        # ===== 4. UPDATE RISK REDUCTION =====
        self.risk_reduction.update()

        # ===== 5. FILTER SAHAM =====
        filtered_stocks = []
        sector_exposure = {}

        # Urutkan berdasarkan Quality Score
        stocks = sorted(stocks, key=lambda x: x['Quality_Score'], reverse=True)

        for stock in stocks:
            if len(filtered_stocks) >= 5:  # Maks 5 saham
                break

            # Cek sektor exposure
            sector = stock['Sector']
            current_exposure = sector_exposure.get(sector, 0)
            if current_exposure >= self.capital_tier.max_sector_pct:
                continue

            # Cek modal
            cost = 100 * stock['Harga']
            if cost > self.remaining_modal:
                continue

            # Cek dengan risk reduction
            can_trade, reason = self.risk_reduction.should_trade(stock)
            if not can_trade:
                continue

            # Cek dengan capital preservation
            can_trade, reason = self.preservation.should_trade(stock)
            if not can_trade:
                continue

            # Hitung ukuran posisi berdasarkan capital tier
            # Estimasi volume harian (simulasi)
            daily_volume_rp = 10000000  # Default Rp 10jt
            if 'Volume' in stock:
                daily_volume_rp = stock.get('avg_volume', 10000000) * stock['Harga']

            base_lot = 1
            recommended_lot = self.capital_tier.get_position_size(
                stock['Harga'], daily_volume_rp, base_lot
            )

            # Apply risk reduction multiplier
            final_lot = self.risk_reduction.get_position_size(recommended_lot)

            if final_lot < 1:
                continue

            # Simulasi eksekusi
            execution_advisor = ExecutionAdvisor()
            expected_price, fill_prob, exec_advice = execution_advisor.estimate_execution_quality(
                stock['Harga'], final_lot, daily_volume_rp
            )

            # Generate insights
            insights = self.decision_intel.generate_insights(stock, self.allocated, exec_advice)

            # Buat copy stock dengan data tambahan
            stock_copy = stock.copy()
            stock_copy['Lot'] = final_lot
            stock_copy['Biaya'] = final_lot * 100 * stock['Harga']
            stock_copy['Alokasi_Persen'] = (stock_copy['Biaya'] / self.modal) * 100
            stock_copy['Fill_Prob'] = fill_prob
            stock_copy['Expected_Exec_Price'] = expected_price
            stock_copy['Insights'] = insights

            filtered_stocks.append(stock_copy)
            self.allocated.append(stock_copy)
            self.remaining_modal -= stock_copy['Biaya']
            sector_exposure[sector] = current_exposure + (stock_copy['Biaya'] / self.modal)
            self.insights.extend(insights)

        # Hitung portfolio risk
        portfolio_risk = 0.2  # Default

        self.allocated = filtered_stocks
        return filtered_stocks, portfolio_risk, self.insights

    def get_summary(self):
        total_invest = self.modal - self.remaining_modal
        total_exp_rp = sum([s['Expected_Return']['rp_per_share'] * s['Lot'] * 100
                           for s in self.allocated if s['Expected_Return']['rp_per_share'] > 0])

        return {
            'total_teralokasi': total_invest,
            'sisa_modal': self.remaining_modal,
            'total_exp_rp': total_exp_rp,
            'total_exp_pct': (total_exp_rp / total_invest * 100) if total_invest > 0 else 0
        }


# ===================== ANALISIS SAHAM (DIUPDATE) =====================
def analyze_stock_parallel(symbol, current_time, model, feature_cols, global_indices,
                          sentiment, model_sharpe=0, returns_cache=None, debug_id=0):
    """Versi analyze_stock dengan tambahan data untuk production"""
    try:
        df_raw = download_stock_data(symbol)
        if df_raw is None:
            return None

        last_price = float(df_raw["Close"].iloc[-1])

        # ===== FILTER HARGA =====
        if last_price < MIN_PRICE or last_price > MAX_PRICE:
            return None

        # ===== FILTER SPREAD =====
        spread_pct = calculate_spread_pct(df_raw)
        max_spread = get_adaptive_max_spread(last_price)
        if spread_pct > max_spread:
            return None

        # ===== FILTER VOLUME =====
        avg_volume = float(df_raw["Volume"].tail(20).mean())
        avg_volume_rp = avg_volume * last_price
        min_volume = get_adaptive_min_volume(last_price)
        if avg_volume_rp < min_volume:
            return None

        # ===== FEATURE ENGINEERING =====
        df_feat = calculate_features_optimized(df_raw)
        if len(df_feat) < 50:
            return None

        # ===== ANALISIS CHART =====
        chart_bias, chart_strength = analyze_chart_bias(df_feat)

        # ===== SEKTOR =====
        sector = get_sector_from_symbol(symbol)

        # ===== SUPPORT/RESISTANCE =====
        try:
            support_levels, resistance_levels = find_historical_support_resistance(df_raw)
            support, support_weight, resistance, resistance_weight = get_nearest_support_resistance(
                last_price, support_levels, resistance_levels
            )
        except:
            support = last_price * 0.95
            support_weight = 0.5
            resistance = last_price * 1.05
            resistance_weight = 0.5

        # ===== INDIKATOR TEKNIKAL =====
        try:
            atr = float(df_feat["ATR"].iloc[-1]) if "ATR" in df_feat.columns else last_price * 0.02
            rsi = float(df_feat["RSI"].iloc[-1]) if "RSI" in df_feat.columns else 50
            macd_hist = float(df_feat["MACD_Hist"].iloc[-1]) if "MACD_Hist" in df_feat.columns else 0
            volume_ratio = float(df_feat["Volume_Ratio"].iloc[-1]) if "Volume_Ratio" in df_feat.columns else 1
        except:
            atr = last_price * 0.02
            rsi = 50
            macd_hist = 0
            volume_ratio = 1

        # ===== MODEL PROBABILITY =====
        model_proba = None
        if model is not None and feature_cols is not None:
            try:
                latest = df_feat[feature_cols].iloc[-1:].fillna(0)
                model_proba = model.predict_proba(latest)[0]
            except:
                pass

        prob_up, prob_down, prob_side = calculate_realistic_probability(df_feat, model_proba)

        # ===== DETERMINE ACTION =====
        action, threshold = determine_action_based_on_sr(
            last_price, support, resistance, prob_up,
            chart_bias, rsi, macd_hist, volume_ratio,
            global_indices, sector, model_sharpe,
            sentiment
        )

        if action is None:
            return None

        # ===== RISK MANAGEMENT =====
        sl_price, tp_price, rr_ratio = calculate_intraday_tp_sl(
            last_price, atr, action, support, resistance, support_weight, resistance_weight
        )

        # Filter RR setelah fee
        if rr_ratio < MIN_RR_RATIO:
            return None

        # ===== EXPECTED RETURN SETELAH FEE =====
        expected_return = calculate_calibrated_expected_return(
            prob_up, prob_down, tp_price, sl_price, last_price, action
        )

        # ===== SIMPAN RETURNS UNTUK KORELASI =====
        stock_returns = None
        if returns_cache is not None and 'Return_1' in df_feat.columns:
            stock_returns = df_feat['Return_1'].dropna().tail(100).values

        # ===== QUALITY SCORE =====
        quality_score = (prob_up * 0.35 +
                        expected_return['pct'] / 20 * 0.25 +
                        chart_strength * 0.15 +
                        (rr_ratio / 3) * 0.15 +
                        (1 - spread_pct / 10) * 0.1)

        if action == "BREAKOUT BUY":
            quality_score *= 1.2
        elif "PULLBACK" in action:
            quality_score *= 1.1

        # ===== CHART SYMBOL =====
        if chart_bias == "BULLISH" and rsi > 50:
            chart_symbol = "🟢"
        elif chart_bias == "BEARISH" or rsi < 30:
            chart_symbol = "🔴"
        else:
            chart_symbol = "🟡"

        # ===== CONFIDENCE INTERVAL =====
        volatility = float(df_feat["Volatility_20"].iloc[-1]) if "Volatility_20" in df_feat.columns else 0.02
        confidence = calculate_confidence_interval(last_price, volatility, prob_up, days=2)

        # ===== TAMBAHKAN DATA UNTUK PRODUCTION =====
        stock_result = {
            "Symbol": symbol.replace(".JK", ""),
            "Harga": round(last_price, 0),
            "Spread_Pct": spread_pct,
            "Support": round(support, 0),
            "Resistance": round(resistance, 0),
            "ATR": round(atr, 0),
            "RSI": round(rsi, 1),
            "Prob_Naik": round(prob_up, 3),
            "Action": action,
            "Sector": sector,
            "Chart_Symbol": chart_symbol,
            "Chart_Bias": chart_bias,
            "Entry": round(last_price, 0),
            "Stop_Loss": sl_price,
            "Take_Profit": tp_price,
            "RR_Ratio": rr_ratio,
            "Expected_Return": expected_return,
            "Confidence": confidence,
            "Min_Lot_Cost": 100 * last_price,
            "Can_Buy": (100 * last_price) <= MODAL,
            "Quality_Score": round(quality_score, 3),
            "returns": stock_returns,
            "avg_volume": avg_volume,
            "daily_volume_rp": avg_volume_rp
        }

        # ================= SPREAD vs EDGE FILTER (ADAPTIVE) =================
        spread_pct = stock_result["Spread_Pct"]
        expected_edge = float(stock_result["Expected_Return"]["pct"])

        # Adaptive tolerance berdasarkan modal
        if MODAL < 100_000:          # MICRO CAPITAL
            tolerance = 2.5
        elif MODAL < 500_000:
            tolerance = 1.8
        elif MODAL < 2_000_000:
            tolerance = 1.2
        else:
            tolerance = 0.8

        # Skip jika friction terlalu ekstrem
        if spread_pct > expected_edge * tolerance:
            return None

        return stock_result

    except Exception as e:
      if DEBUG_MODE:
          print(f"Error analyzing {symbol}: {str(e)[:50]}")
      return None


# ===================== ANALISIS BATCH PARALLEL =====================
def analyze_stocks_batch(all_stocks, current_time, model, feature_cols, global_indices, sentiment, model_sharpe):
    """Analisis semua saham secara parallel."""
    results = []
    successful = 0

    print(f"\n🔍 Menganalisis {len(all_stocks)} saham...")

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_symbol = {}
        debug_id = 0
        for symbol in all_stocks:
            future = executor.submit(analyze_stock_parallel, symbol, current_time,
                                    model, feature_cols, global_indices, sentiment,
                                    model_sharpe, None, debug_id)
            future_to_symbol[future] = symbol
            debug_id += 1

        completed = 0
        for future in as_completed(future_to_symbol):
            try:
                result = future.result()
                if result is not None:
                    with results_lock:
                        results.append(result)
                        successful += 1
            except Exception as e:
                pass

            completed += 1
            if completed % 50 == 0:
                print(f"  Progress: {completed}/{len(all_stocks)} saham ({successful} ditemukan)")

    print(f"✅ Selesai! {successful} saham ditemukan")
    return results


# ===================== DAFTAR SAHAM IDX =====================
def get_active_idx_stocks() -> list:
    return [
        "AALI.JK", "ABBA.JK", "ABDA.JK", "ABMM.JK", "ACES.JK", "ADES.JK",
        "ADHI.JK", "ADMF.JK", "ADMG.JK", "ADRO.JK", "AGRO.JK", "AKRA.JK",
        "AKSI.JK", "ALDO.JK", "ALKA.JK", "AMAG.JK", "AMFG.JK", "AMRT.JK",
        "ANTM.JK", "APEX.JK", "APIC.JK", "APLN.JK", "ARCI.JK", "ARGO.JK",
        "ARII.JK", "ARNA.JK", "ARTA.JK", "ARTO.JK", "ASBI.JK", "ASDM.JK",
        "ASGR.JK", "ASII.JK", "ASMI.JK", "ASPI.JK", "ASRM.JK", "ASSA.JK",
        "AUTO.JK", "BABP.JK", "BACA.JK", "BALI.JK", "BANK.JK", "BAPA.JK",
        "BAPI.JK", "BATA.JK", "BAYU.JK", "BBCA.JK", "BBHI.JK", "BBKP.JK",
        "BBLD.JK", "BBNI.JK", "BBRI.JK", "BBRM.JK", "BBTN.JK", "BBYB.JK",
        "BCAP.JK", "BCIC.JK", "BDMN.JK", "BEEF.JK", "BEKS.JK", "BEST.JK",
        "BFIN.JK", "BGTG.JK", "BHAT.JK", "BIMA.JK", "BINA.JK", "BIPP.JK",
        "BIRD.JK", "BISI.JK", "BJBR.JK", "BJTM.JK", "BKSW.JK", "BLTA.JK",
        "BLUE.JK", "BMAS.JK", "BMRI.JK", "BMSR.JK", "BMTR.JK", "BNBA.JK",
        "BNGA.JK", "BNII.JK", "BNLI.JK", "BOBA.JK", "BOLT.JK", "BOSS.JK",
        "BPFI.JK", "BPII.JK", "BPTR.JK", "BRAM.JK", "BRIS.JK", "BRMS.JK",
        "BRNA.JK", "BRPT.JK", "BSDE.JK", "BSIM.JK", "BSSR.JK", "BTEL.JK",
        "BTON.JK", "BTPN.JK", "BTPS.JK", "BUDI.JK", "BUMI.JK", "BUVA.JK",
        "BWPT.JK", "BYAN.JK", "CAMP.JK", "CANI.JK", "CASA.JK", "CASS.JK",
        "CBMF.JK", "CCSI.JK", "CDAX.JK", "CEKA.JK", "CENT.JK", "CFIN.JK",
        "CITA.JK", "CITY.JK", "CKRA.JK", "CLEO.JK", "CLPI.JK", "CMNP.JK",
        "CMPP.JK", "CMRY.JK", "CNKO.JK", "CNTX.JK", "COAL.JK", "COCO.JK",
        "COWL.JK", "CPIN.JK", "CPRO.JK", "CSAP.JK", "CSIS.JK", "CTBN.JK",
        "CTRA.JK", "CTTH.JK", "DART.JK", "DASA.JK", "DAYA.JK", "DCII.JK",
        "DEGA.JK", "DEWA.JK", "DFAM.JK", "DGIK.JK", "DIGI.JK", "DILD.JK",
        "DIVN.JK", "DKFT.JK", "DLTA.JK", "DMAS.JK", "DMND.JK", "DNET.JK",
        "DNLS.JK", "DOID.JK", "DPNS.JK", "DPUM.JK", "DSFI.JK", "DSNG.JK",
        "DUCK.JK", "DUTI.JK", "DVLA.JK", "DYAN.JK", "EASI.JK", "EASY.JK",
        "EBMT.JK", "ECII.JK", "EDGE.JK", "EKAD.JK", "ELBA.JK", "ELSA.JK",
        "ELTY.JK", "EMBR.JK", "EMDE.JK", "EMTK.JK", "ENRG.JK", "ENVY.JK",
        "EPAC.JK", "EPMT.JK", "ERAA.JK", "ERTX.JK", "ESSA.JK", "ESTA.JK",
        "ESTI.JK", "ETWA.JK", "EXCL.JK", "FAST.JK", "FASW.JK", "FISH.JK",
        "FITT.JK", "FKSF.JK", "FLMC.JK", "FMII.JK", "FORU.JK", "FORZ.JK",
        "FPNI.JK", "FREN.JK", "FUJI.JK", "FUTR.JK", "GAMA.JK", "GDST.JK",
        "GDYR.JK", "GEMS.JK", "GGRM.JK", "GGRP.JK", "GHON.JK", "GIDS.JK",
        "GJTL.JK", "GLVA.JK", "GMFI.JK", "GMTD.JK", "GOLD.JK", "GOOD.JK",
        "GPRA.JK", "GSMF.JK", "GTBO.JK", "GTSI.JK", "GULA.JK", "GZCO.JK",
        "HADE.JK", "HDFA.JK", "HDIT.JK", "HEAL.JK", "HERO.JK", "HITS.JK",
        "HKMU.JK", "HMSP.JK", "HOKI.JK", "HOMI.JK", "HOPE.JK", "HOTL.JK",
        "HRME.JK", "HRTA.JK", "HRUM.JK", "HSBK.JK", "HSMP.JK", "HUMI.JK",
        "IBFN.JK", "IBST.JK", "ICBP.JK", "ICON.JK", "IDPR.JK", "IFII.JK",
        "IFSH.JK", "IGAR.JK", "IIKP.JK", "IKAI.JK", "IKAN.JK", "IMAS.JK",
        "IMJS.JK", "IMPC.JK", "INAF.JK", "INAI.JK", "INCF.JK", "INCI.JK",
        "INCO.JK", "INDF.JK", "INDX.JK", "INDY.JK", "INKP.JK", "INPC.JK",
        "INPP.JK", "INPS.JK", "INRU.JK", "INTA.JK", "INTD.JK", "INTP.JK",
        "IPCC.JK", "IPCM.JK", "IPOL.JK", "ISAT.JK", "ISSP.JK", "ITIC.JK",
        "ITMG.JK", "JACK.JK", "JAST.JK", "JAWA.JK", "JECC.JK", "JEMB.JK",
        "JFAS.JK", "JGLE.JK", "JHON.JK", "JIHD.JK", "JKON.JK", "JKSW.JK",
        "JMAS.JK", "JPFA.JK", "JPII.JK", "JPUR.JK", "JRPT.JK", "JSKY.JK",
        "JSMR.JK", "JSPT.JK", "JTNB.JK", "KAEF.JK", "KARW.JK", "KBLI.JK",
        "KBLM.JK", "KBRT.JK", "KBRI.JK", "KDSI.JK", "KDTN.JK", "KEEN.JK",
        "KETR.JK", "KIJA.JK", "KINO.JK", "KIOS.JK", "KJEN.JK", "KKGI.JK",
        "KLBF.JK", "KMTR.JK", "KOBX.JK", "KOIN.JK", "KOLI.JK", "KONI.JK",
        "KPAL.JK", "KPIG.JK", "KRAS.JK", "KREN.JK", "KRYA.JK", "KSEL.JK",
        "KUAS.JK", "KUIC.JK", "KUVO.JK", "LAPD.JK", "LATA.JK", "LBAK.JK",
        "LCGP.JK", "LCKM.JK", "LEAD.JK", "LIFE.JK", "LINK.JK", "LION.JK",
        "LISA.JK", "LMAS.JK", "LMPI.JK", "LMSH.JK", "LPCK.JK", "LPGI.JK",
        "LPIN.JK", "LPKR.JK", "LPLI.JK", "LPPF.JK", "LPPS.JK", "LSIP.JK",
        "LSPI.JK", "LTLS.JK", "LUCY.JK", "MABA.JK", "MABH.JK", "MAGP.JK",
        "MAIN.JK", "MAMI.JK", "MAPA.JK", "MAPB.JK", "MAPI.JK", "MARA.JK",
        "MASA.JK", "MAYA.JK", "MBAP.JK", "MBCA.JK", "MBSS.JK", "MBTO.JK",
        "MCAS.JK", "MCPI.JK", "MCOR.JK", "MDIA.JK", "MDKA.JK", "MDKI.JK",
        "MEDC.JK", "MEGA.JK", "MERK.JK", "META.JK", "MFIN.JK", "MFMI.JK",
        "MGNA.JK", "MGRO.JK", "MIKA.JK", "MINA.JK", "MIRA.JK", "MITI.JK",
        "MITT.JK", "MKNT.JK", "MKPI.JK", "MLBI.JK", "MLIA.JK", "MLPL.JK",
        "MLPT.JK", "MLSL.JK", "MMIX.JK", "MMLP.JK", "MNBA.JK", "MNCN.JK",
        "MOLI.JK", "MPOW.JK", "MPPA.JK", "MPRO.JK", "MPTJ.JK", "MRAT.JK",
        "MSIE.JK", "MSIN.JK", "MTDL.JK", "MTFN.JK", "MTLA.JK", "MTPS.JK",
        "MTSM.JK", "MUDA.JK", "MUTU.JK", "MYOH.JK", "MYOR.JK", "MYRX.JK",
        "MYSX.JK", "NAGA.JK", "NATO.JK", "NELY.JK", "NFCX.JK", "NIKL.JK",
        "NISP.JK", "NITY.JK", "NIYM.JK", "NOBU.JK", "NPGF.JK", "NRCA.JK",
        "NUSA.JK", "NUSI.JK", "OASA.JK", "OCTN.JK", "OKAS.JK", "OMED.JK",
        "ONIX.JK", "OPMS.JK", "ORNA.JK", "OTBK.JK", "PADI.JK", "PAMG.JK",
        "PANR.JK", "PANS.JK", "PANU.JK", "PAPA.JK", "PASA.JK", "PASS.JK",
        "PBRX.JK", "PBID.JK", "PBSA.JK", "PCAR.JK", "PDES.JK", "PDGD.JK",
        "PDIN.JK", "PEGE.JK", "PGLI.JK", "PGUN.JK", "PICO.JK", "PIDRA.JK",
        "PJAA.JK", "PKPK.JK", "PLAN.JK", "PLAS.JK", "PLIN.JK", "PMJS.JK",
        "PMMP.JK", "PNBN.JK", "PNBS.JK", "PNIN.JK", "PNLF.JK", "PNSE.JK",
        "POLL.JK", "POLU.JK", "POLY.JK", "POOL.JK", "PORT.JK", "POWR.JK",
        "PPGL.JK", "PPRE.JK", "PPRO.JK", "PPSI.JK", "PRAS.JK", "PRDA.JK",
        "PRIM.JK", "PRIN.JK", "PRLD.JK", "PROD.JK", "PROT.JK", "PRTS.JK",
        "PSAB.JK", "PSBA.JK", "PSDN.JK", "PSGO.JK", "PSKT.JK", "PSSI.JK",
        "PTBA.JK", "PTDU.JK", "PTIS.JK", "PTMP.JK", "PTPP.JK", "PTPW.JK",
        "PTRO.JK", "PTSN.JK", "PUDP.JK", "PURA.JK", "PURE.JK", "PWON.JK",
        "PYFA.JK", "RACE.JK", "RADIO.JK", "RAFI.JK", "RAJA.JK", "RAKD.JK",
        "RALS.JK", "RANC.JK", "RATU.JK", "RBMS.JK", "RDTX.JK", "REAL.JK",
        "RELI.JK", "RIGS.JK", "RIMO.JK", "RISE.JK", "RMBA.JK", "RODA.JK",
        "ROKI.JK", "ROTI.JK", "RRMI.JK", "RUIS.JK", "RUMI.JK", "SABA.JK",
        "SAFE.JK", "SAME.JK", "SAPX.JK", "SARA.JK", "SATO.JK", "SBAT.JK",
        "SBBP.JK", "SBGA.JK", "SBMA.JK", "SBMF.JK", "SCBD.JK", "SCCC.JK",
        "SCCO.JK", "SCMA.JK", "SDPC.JK", "SDRA.JK", "SEAN.JK", "SECR.JK",
        "SEMA.JK", "SFAN.JK", "SGRO.JK", "SHID.JK", "SHIP.JK", "SIDO.JK",
        "SILO.JK", "SIMA.JK", "SIMP.JK", "SIPD.JK", "SIPO.JK", "SKBM.JK",
        "SKLT.JK", "SKRN.JK", "SLIS.JK", "SMAR.JK", "SMDR.JK", "SMGR.JK",
        "SMIL.JK", "SMMT.JK", "SMSM.JK", "SMRA.JK", "SNLK.JK", "SNMS.JK",
        "SOFA.JK", "SONA.JK", "SOSS.JK", "SPMA.JK", "SPMI.JK", "SPNA.JK",
        "SPTO.JK", "SQBI.JK", "SQMI.JK", "SRAJ.JK", "SRIL.JK", "SRSN.JK",
        "SSIA.JK", "SSMS.JK", "SSTM.JK", "STAR.JK", "STTP.JK", "SUGI.JK",
        "SULI.JK", "SUPR.JK", "SURI.JK", "SWAT.JK", "TALD.JK", "TAMA.JK",
        "TAMU.JK", "TARA.JK", "TASP.JK", "TATA.JK", "TAXI.JK", "TBIG.JK",
        "TBLA.JK", "TCID.JK", "TDPM.JK", "TELE.JK", "TEMB.JK", "TEMPO.JK",
        "TIFA.JK", "TIGA.JK", "TINS.JK", "TIRA.JK", "TIRT.JK", "TITA.JK",
        "TKGA.JK", "TKIM.JK", "TLKM.JK", "TMAS.JK", "TMPO.JK", "TMSH.JK",
        "TOBA.JK", "TOPS.JK", "TOTL.JK", "TOTO.JK", "TOWR.JK", "TPIA.JK",
        "TPMA.JK", "TRAM.JK", "TRGU.JK", "TRIO.JK", "TRIS.JK", "TRJA.JK",
        "TRST.JK", "TRUB.JK", "TRUK.JK", "TRUS.JK", "TSPC.JK", "TUGU.JK",
        "TURI.JK", "TUVN.JK", "TYRE.JK", "UCID.JK", "UDIJ.JK", "UFNX.JK",
        "UGRO.JK", "UJSN.JK", "ULTJ.JK", "UNIC.JK", "UNIQ.JK", "UNIT.JK",
        "UNSP.JK", "UNTR.JK", "UNVR.JK", "USFI.JK", "VALU.JK", "VICO.JK",
        "VIDI.JK", "VIVA.JK", "VOKS.JK", "VRNA.JK", "VTNY.JK", "WAPO.JK",
        "WEGE.JK", "WEHA.JK", "WICO.JK", "WIFI.JK", "WIIM.JK", "WIKA.JK",
        "WINS.JK", "WMPP.JK", "WOOD.JK", "WOWS.JK", "WRKR.JK", "WSBP.JK",
        "WSKT.JK", "WTON.JK", "YELO.JK", "YULE.JK", "ZBRA.JK", "ZINC.JK",
        "ZONE.JK"
    ]


# ===================== MAIN PROGRAM (PRODUCTION VERSION) =====================
def main():
    current_time = datetime.now()

    # Ambil data indeks global
    global_indices = get_global_indices_data()

    # Hitung sentimen pasar
    sentiment = get_market_sentiment(global_indices)

    print("=" * 120)
    print("🏦 SAHAM IDX TRADING SYSTEM - PRODUCTION READY VERSION")
    print("=" * 120)
    print(f"📅 Tanggal: {current_time.strftime('%Y-%m-%d %H:%M')} WIB")
    print(f"💰 Modal: Rp {MODAL:,.0f}")
    print(f"📊 Kategori Harga: {PRICE_CATEGORY}")
    print(f"📊 Filter: Harga Rp{MIN_PRICE}-{MAX_PRICE}")
    print(f"📊 Fee Transaksi: {FEE_BUY*100:.2f}% beli + {FEE_SELL*100:.2f}% jual = {TOTAL_FEE*100:.2f}%")
    print(f"📊 Max per Saham: {MAX_POSITION_SIZE*100:.0f}%")
    print(f"🎯 Min Risk/Reward (after fee): 1:{MIN_RR_RATIO}")
    print(f"⚡ Walk-forward validation with Sharpe ratio")
    print(f"📈 Confidence Intervals (bukan path prediction)")
    print(f"🛡️ Risk Management: ARR + Cooldown + Kill Switch + Capital Preservation")
    print("-" * 120)

    # Tampilkan status indeks global
    print("\n🌍 STATUS INDEKS GLOBAL")
    print("-" * 120)

    important_indices = ["IHSG", "DOWJONES", "HANGSENG", "OIL", "GOLD", "USDIDR"]
    for name in important_indices:
        if name in global_indices and global_indices[name]["value"] > 0:
            data = global_indices[name]
            print(f"  {name:<10} {data['value']:>12,.2f} {data['change']:>+8.2f}% {data['momentum']:>+8.2f}% {data['trend']}")

    print("\n📊 SENTIMEN PASAR")
    print("-" * 120)
    print(f"  Kondisi: {sentiment['condition']}")
    print(f"  Volatilitas: {sentiment.get('volatility_regime', 'NORMAL')}")
    for detail in sentiment['details']:
        print(f"    • {detail}")
    print(f"  Rekomendasi: {sentiment['recommendation']}")
    print("-" * 120)

    # Inisialisasi komponen production
    capital_tier = CapitalTierDetector(MODAL)
    trade_history = TradeHistoryTracker()
    risk_reduction = AutomaticRiskReduction(trade_history)
    cooldown = TradingCooldown()
    preservation = CapitalPreservationMode(MODAL)
    kill_switch = KillSwitch(MODAL)
    health_monitor = ModelHealthMonitor()
    decision_intel = DecisionIntelligence(capital_tier.tier, trade_history, sentiment)

    print(f"\n💰 Capital Tier: {capital_tier.tier}")
    print(f"  Max Position: {capital_tier.max_position_pct*100:.0f}% per saham")
    print(f"  Max Sector: {capital_tier.max_sector_pct*100:.0f}% per sektor")

    # Training model dengan walk-forward validation
    print("\n🎯 Melatih model dengan walk-forward validation...")

    all_stocks = get_active_idx_stocks()
    sample_size = min(100, len(all_stocks))
    sample_stocks = np.random.choice(all_stocks, sample_size, replace=False)

    train_features = []
    train_targets = []
    train_prices = []
    feature_cols = None
    successful_samples = 0

    for symbol in sample_stocks:
        try:
            df_raw = download_stock_data(symbol)
            if df_raw is None:
                continue

            last_price = float(df_raw["Close"].iloc[-1])
            if last_price < MIN_PRICE or last_price > MAX_PRICE:
                continue

            df_feat = calculate_features_optimized(df_raw)
            if len(df_feat) < 50:
                continue

            df_model = df_feat[df_feat["Target"].notna()].copy()
            if len(df_model) < 30:
                continue

            if feature_cols is None:
                feature_cols = [c for c in df_feat.columns if c not in ["Target", "Future_Return", "Return_Vol_Ratio", "Volatility_Norm"]]

            # Filter kolom yang ada
            available_cols = [c for c in feature_cols if c in df_model.columns]

            train_features.append(df_model[available_cols].fillna(0))
            train_targets.append(df_model["Target"])
            train_prices.append(df_model["Close"])
            successful_samples += 1

        except Exception as e:
            continue

    if len(train_features) == 0:
        print("❌ Tidak cukup data training, menggunakan rule-based system")
        best_model = None
        best_sharpe = 0.5
        feature_cols = None
    else:
        X_train = pd.concat(train_features, ignore_index=True)
        y_train = pd.concat(train_targets, ignore_index=True)
        prices_train = pd.concat(train_prices, ignore_index=True)

        print(f"✅ Data training: {len(X_train)} samples dari {successful_samples} saham")
        print(f"   Distribusi target: {np.bincount(y_train.astype(int))}")

        # Evaluasi model dengan walk-forward validation
        models = get_model_candidates()
        evaluator = ModelEvaluator(n_splits=N_SPLITS)
        evaluator.evaluate(models, X_train, y_train, prices_train)
        evaluator.print_results()

        # Dapatkan Sharpe terbaik
        best_sharpe = evaluator.best_sharpe_value
        best_model = evaluator.get_best_model_trained(X_train, y_train)

        print(f"\n📊 Best Model Sharpe: {best_sharpe:.3f}")
        if best_sharpe > 1.0:
            print("✅ Model SANGAT BAIK untuk trading")
        elif best_sharpe > 0.5:
            print("⚠️ Model CUKUP BAIK")
        elif best_sharpe > 0:
            print("📊 Model VALID (positif)")
        else:
            print("❌ Model TIDAK VALID, menggunakan rule-based")
            best_model = None

    # Analisis semua saham
    results = analyze_stocks_batch(all_stocks, current_time, best_model, feature_cols,
                                   global_indices, sentiment, best_sharpe if best_model else 0.5)

    if not results:
        print("\n❌ Tidak ada saham yang memenuhi kriteria!")
        print("\n💡 SARAN:")
        print("  • Coba jalankan lagi nanti")
        print("  • Periksa koneksi internet")
        print("  • Jika terus terjadi, turunkan filter volume/spread")
        return

    # Urutkan berdasarkan Quality Score
    results = sorted(results, key=lambda x: x['Quality_Score'], reverse=True)

    # ===================== PORTFOLIO =====================
    print("\n" + "=" * 120)
    print(f"📊 PORTFOLIO REKOMENDASI (Modal Rp {MODAL:,.0f})")
    print("=" * 120)
    print(f"🎯 Fokus pada saham dengan harga {PRICE_CATEGORY}")
    print(f"📊 Sentimen Pasar: {sentiment['condition']}")
    print(f"📊 Fee Transaksi: {TOTAL_FEE*100:.2f}% (sudah termasuk dalam perhitungan)")

    buyable = [s for s in results if s['Can_Buy']]

    if not buyable:
        print(f"\n❌ Tidak ada saham yang bisa dibeli dengan modal Rp {MODAL:,.0f}!")
        print(f"   Harga termurah: Rp {min([s['Harga'] for s in results]) * 100:,.0f} per lot")
        print("\n📊 DAFTAR SAHAM TERBAIK (TIDAK BISA DIBELI):")

        display_data = []
        for stock in results[:10]:
            conf = stock['Confidence']
            display_data.append([
                stock['Symbol'],
                stock['Sector'],
                f"Rp {stock['Harga']:,.0f}",
                f"{stock['RSI']:.0f}",
                stock['Action'],
                f"{stock['Prob_Naik']:.0%}",
                f"1:{stock['RR_Ratio']}",
                f"{stock['Expected_Return']['pct']}%",
                f"Rp {conf['ci_68'][0]:,.0f}-{conf['ci_68'][1]:,.0f}",
                f"Rp {stock['Min_Lot_Cost']:,.0f}"
            ])

        headers = ["Kode", "Sektor", "Harga", "RSI", "Action", "Prob", "R/R", "Exp%", "CI 68%", "Biaya 1 Lot"]
        print(tabulate(display_data, headers=headers, tablefmt="grid"))
        return

    # Alokasi dengan production portfolio allocator
    allocator = ProductionPortfolioAllocator(
        MODAL, capital_tier, risk_reduction, cooldown,
        preservation, kill_switch, health_monitor, decision_intel
    )

    portfolio, portfolio_risk, all_insights = allocator.allocate(buyable, current_time)

    if not portfolio:
        print("\n❌ Gagal mengalokasikan modal karena risk management!")
        return

    # ===================== TOP 3 PORTFOLIO =====================
    top_3 = portfolio[:3]

    print("\n" + "=" * 120)
    print("📈 TOP 3 SAHAM TERBAIK DALAM PORTFOLIO")
    print("=" * 120)

    for i, stock in enumerate(top_3, 1):
        conf = stock['Confidence']
        print(f"\n{i}. {stock['Symbol']} ({stock['Sector']}) - Harga: Rp {stock['Harga']:,.0f} | RSI: {stock['RSI']:.0f}")
        print(f"   📊 Support: Rp {stock['Support']:,.0f} | Resistance: Rp {stock['Resistance']:,.0f}")
        print(f"   📊 Action: {stock['Action']} | Entry: Rp {stock['Entry']:,.0f}")
        print(f"   🛑 Stop Loss: Rp {stock['Stop_Loss']:,.0f} | 🎯 Take Profit: Rp {stock['Take_Profit']:,.0f}")
        print(f"   📈 Risk/Reward (after fee): 1:{stock['RR_Ratio']} | Expected Return: {stock['Expected_Return']['pct']}%")
        print(f"   📊 Confidence Interval 68%: Rp {conf['ci_68'][0]:,.0f} - Rp {conf['ci_68'][1]:,.0f}")
        print(f"   📈 Expected Price 2 Hari: Rp {conf['expected_price']:,.0f} ({conf['expected_return']})")
        print(f"   📊 Fill Probability: {stock.get('Fill_Prob', 0):.0%}")

    # ===================== TABEL PORTFOLIO LENGKAP =====================
    print("\n" + "=" * 120)
    print("📊 RINCIAN SELURUH PORTFOLIO")
    print("=" * 120)

    display_data = []
    for stock in portfolio:
        exp_rp = stock['Expected_Return']['rp_per_share'] * stock['Lot'] * 100
        conf = stock['Confidence']

        display_data.append([
            stock['Symbol'],
            stock['Sector'],
            f"Rp {stock['Harga']:,.0f}",
            f"{stock['Spread_Pct']:.1f}%",
            f"Rp {stock['Support']:,.0f}",
            f"Rp {stock['Resistance']:,.0f}",
            f"{stock['RSI']:.0f}",
            stock['Chart_Symbol'],
            stock['Action'],
            f"Rp {stock['Entry']:,.0f}",
            f"Rp {stock['Stop_Loss']:,.0f}",
            f"Rp {stock['Take_Profit']:,.0f}",
            f"1:{stock['RR_Ratio']}",
            f"{stock['Expected_Return']['pct']}%",
            f"Rp {exp_rp:,.0f}",
            f"{conf['expected_price']:,.0f}",
            f"{conf['ci_68'][0]:,.0f}-{conf['ci_68'][1]:,.0f}",
            f"{conf['prob_up']:.0%}",
            f"{stock.get('Fill_Prob', 0):.0%}",
            stock['Lot'],
            f"Rp {stock['Biaya']:,.0f}",
            f"{stock['Alokasi_Persen']:.1f}%"
        ])

    headers = ["Kode", "Sektor", "Harga", "Sprd", "Supp", "Res", "RSI", "", "Action",
              "Entry", "SL", "TP", "R/R", "Exp%", "Exp Rp", "Pred 2H", "CI 68%", "Prob", "Fill", "Lot", "Biaya", "Alokasi"]

    print(tabulate(display_data, headers=headers, tablefmt="grid"))

    summary = allocator.get_summary()
    print(f"\n📊 Total Investasi: Rp {summary['total_teralokasi']:,.0f}")
    print(f"📊 Sisa Modal: Rp {summary['sisa_modal']:,.0f}")
    print(f"📊 Expected Return (after fee): Rp {summary['total_exp_rp']:,.0f} ({summary['total_exp_pct']:.1f}%)")
    print(f"📊 Portfolio Risk (estimasi): {portfolio_risk:.3f}")

    # ===================== DECISION INSIGHTS =====================
    print("\n" + "=" * 120)
    print("🧠 DECISION INSIGHTS")
    print("=" * 120)

    # Tampilkan insights per saham
    for stock in portfolio[:3]:
        if 'Insights' in stock and stock['Insights']:
            print(f"\n{stock['Symbol']}:")
            for insight in stock['Insights'][:3]:  # Max 3 insights per saham
                print(f"  {insight}")

    # Tampilkan market summary
    market_summary = decision_intel.generate_market_summary()
    if market_summary:
        print(f"\n📊 Market Summary:")
        for line in market_summary:
            print(f"  {line}")

    # Tampilkan trading tips
    tips = decision_intel.generate_trading_tips()
    if tips:
        print(f"\n💡 Trading Tips:")
        for tip in tips:
            print(f"  {tip}")

    # Tampilkan risk status
    print(f"\n🛡️ Risk Status:")
    print(f"  • Mode: {preservation.mode}")
    print(f"  • Position Multiplier: {risk_reduction.position_multiplier:.1f}x")
    print(f"  • Consecutive Losses: {trade_history.get_metrics().get('consecutive_losses', 0)}")

    if cooldown.cooldown_active:
        remaining = (cooldown.cooldown_until - current_time).total_seconds() / 3600
        print(f"  • Cooldown: ACTIVE ({cooldown.cooldown_reason}) - {remaining:.1f} jam")

    # ===================== STATISTIK =====================
    print("\n" + "=" * 120)
    print("📊 STATISTIK")
    print("=" * 120)

    avg_prob = np.mean([s['Prob_Naik'] for s in results])
    avg_rr = np.mean([s['RR_Ratio'] for s in results])
    avg_exp = np.mean([float(s['Expected_Return']['pct']) for s in results])

    print(f"Total Saham Dianalisis      : {len(all_stocks)}")
    print(f"Saham Ditemukan             : {len(results)}")
    print(f"Saham Bisa Dibeli           : {len(buyable)}")
    print(f"Saham dalam Portfolio       : {len(portfolio)}")
    print(f"Rata-rata Probabilitas      : {avg_prob:.1%}")
    print(f"Rata-rata Risk/Reward       : 1:{avg_rr:.2f}")
    print(f"Rata-rata Expected Return   : {avg_exp:.2f}%")
    print(f"Model Sharpe Ratio          : {best_sharpe if best_model else 0.5:.3f}")

    print("\n" + "=" * 120)
    print("⚠️  DISCLAIMER")
    print("=" * 120)
    print("• Hasil ini berdasarkan analisis teknikal dan machine learning")
    print("• Semua perhitungan sudah termasuk fee transaksi 0.3%")
    print("• Confidence interval adalah estimasi statistik, bukan prediksi pasti")
    print("• Portfolio risk dihitung berdasarkan korelasi antar saham")
    print("• Model divalidasi dengan walk-forward method menggunakan Sharpe ratio")
    print("• Risk management: ARR, Cooldown, Kill Switch, Capital Preservation aktif")
    print("• Decision insights hanya panduan, keputusan akhir tetap di tangan trader")
    print("• Bukan rekomendasi beli/jual - lakukan riset mandiri")
    print("• 1 lot = 100 lembar saham (minimal pembelian di BEI)")
    print("=" * 120)

Masukkan modal Anda (Rp): 40000


In [39]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n⚠️ Program dihentikan oleh user")
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

📡 Mengunduh data indeks global (sekali saja)...
🏦 SAHAM IDX TRADING SYSTEM - PRODUCTION READY VERSION
📅 Tanggal: 2026-03-01 21:16 WIB
💰 Modal: Rp 40,000
📊 Kategori Harga: RENDAH (Rp 10-500)
📊 Filter: Harga Rp10-500
📊 Fee Transaksi: 0.15% beli + 0.15% jual = 0.30%
📊 Max per Saham: 50%
🎯 Min Risk/Reward (after fee): 1:1.5
⚡ Walk-forward validation with Sharpe ratio
📈 Confidence Intervals (bukan path prediction)
🛡️ Risk Management: ARR + Cooldown + Kill Switch + Capital Preservation
------------------------------------------------------------------------------------------------------------------------

🌍 STATUS INDEKS GLOBAL
------------------------------------------------------------------------------------------------------------------------
  IHSG           8,235.49    -0.04%    +0.72% 🟢 BULLISH
  DOWJONES      48,975.64    +0.11%    -0.05% 🟢 BULLISH
  HANGSENG      26,630.54    -0.10%    +0.34% 🟢 BULLISH
  OIL               67.02    -0.34%    +0.72% 🔴 BEARISH
  GOLD           5,247.90